Included installs

In [1]:
# !pip install pandas

In [2]:
# !pip install influxdb

In [3]:
# !pip install matplotlib

In [4]:
# !pip install fuzzywuzzy

In [5]:
# !pip install openpyxl

In [6]:
# !pip install lxml

Not included installs, but useful

In [7]:
# !pip install feedparser

In [8]:
# !pip install lxml

In [9]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json
# import feedparser

Set params

In [10]:
purge=False
# purge=True

In [11]:
process_annoations=False
# process_annoations=True

In [12]:
# process_macro=False
process_macro=True

In [13]:
# write_out=False
write_out=True

In [14]:
full_overwrite=False
# full_overwrite=True

Load pre-params

In [15]:
df0=pd.read_csv('df0.csv').set_index('date')
df0.index=pd.to_datetime(df0.index)

In [16]:
bets=json.loads(open('bets.json','r').read())

In [17]:
severity=pd.read_csv('severity.csv').set_index('date')
severity.index=pd.to_datetime(severity.index)

In [18]:
mobility=pd.read_csv('mobility.csv').set_index('date')
mobility.index=pd.to_datetime(mobility.index)

In [19]:
mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')
mobility_mini.index=pd.to_datetime(mobility_mini.index)

Establish DB connection

In [20]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
dbname_long='long'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)
client_long = DataFrameClient(host, port, user, password, dbname_long)

In [21]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '600d', 1, default=True)
    client_long.drop_database(dbname_long)
    client_long.drop_retention_policy(dbname_long)
    client_long.create_database(dbname_long)
    client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [22]:
htmlipath='../html/'
htmlepath='//myserv-html.er/'
htmlepath_other='//mybackupserv-html.er/'

In [23]:
titles={'HU':"Magyar",'RO':'Română','EN':'English'}

In [24]:
rtitles={titles[t]:t for t in titles}

In [25]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [26]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [27]:
folder_id=[f['id'] for f in folders if f['title']=='My Grafana Folder'][0] #General

In [28]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs=json.loads(response.content)

In [29]:
uids={rtitles[d['title']]:d['uid'] for d in dashs if d['title'] in rtitles}
iids={rtitles[d['title']]:d['id'] for d in dashs if d['title'] in rtitles}

In [30]:
uids_light={rtitles[d['title'].split(' Light')[0]]:d['uid'] for d in dashs if 'Light' in d['title']}
iids_light={rtitles[d['title'].split(' Light')[0]]:d['id'] for d in dashs if 'Light' in d['title']}

In [31]:
uids

{'EN': 'en', 'HU': 'hu', 'RO': 'ro'}

In [33]:
languages=['HU','RO','EN']

In [34]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [35]:
def get_szotar():
    sheet='szotar'
    columns=languages+[i+'_description' for i in languages]+[i+'_source' for i in languages]
    df=pd.read_csv(url+sheet)
    df=df[['ID','UI']+columns]
    sheet='minidashboard'
    df2=pd.read_csv(url+sheet)
    df2=df2[['ID','UI']+columns]
    df=pd.concat([df,df2])
    szotardf=df.set_index('ID')[columns]
    szotar=df.set_index('ID').T.to_dict()
    szotarHU=df.set_index('HU',drop=False).T.to_dict()
    szotarRO=df.set_index('RO',drop=False).T.to_dict()
    szotarEN=df.set_index('EN',drop=False).T.to_dict()
    return szotardf,szotar,szotarHU,szotarRO,szotarEN

In [36]:
def resolve_time_conflicts(series):
    ds={}
    ts=[]
    for d in series:
        if d not in ds:
            ds[d]=pd.to_datetime(d)
            t=(ds[d])
        else:
            ds[d]=ds[d]+pd.to_timedelta('193m')
            t=(ds[d])
        ts.append(t)
    return ts

# Influx

In [37]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-35-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-35-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-35-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [38]:
import time
import datetime
import pytz
utc=pytz.UTC

In [39]:
def push2influx(df,measurement,field_columns,tag_columns,shift=False,dbclient=client,wo=write_out,fo=full_overwrite,daily=True):
    if wo:
        df=df.sort_index()
        # df.index = utc.localize(df.index) 
        df.index=df.index.tz_localize('GMT')#.tz_convert('Europe/Bucharest')
        if shift:
            df.index+=pd.to_timedelta('12h')
        if fo: 
            print('Purging',measurement,'...')
            dbclient.query('DROP MEASUREMENT '+measurement)
        else:
            latest=dbclient.query('SELECT * FROM '+measurement+' GROUP BY "1d" ORDER BY DESC LIMIT 1')
            if latest:
                lat=latest[list(latest.keys())[0]].index[0]
                if daily: lat+=pd.to_timedelta('1d')
                df=df[lat:]
                print('Slicing',measurement,'from',lat,'...')
            else:
                print('No data in db for',measurement,'...')
        time.sleep(3)
        print('Writing to',measurement,'...')
        bsize=5000
        bwait=2
        print(len(df),'data points will be written in',len(df)/bsize,'batches.')
        print('Expected query running time is:',int((len(df)/bsize)*bwait*1.1)+3,'seconds.')
        for i in range(int(len(df)/bsize)+1):
            r=range(i*bsize,min(len(df),(i+1)*bsize))
            dc=df.iloc[r]
            print('Writing batch',i+1,'...')
            dbclient.write_points(dc, measurement, protocol=protocol,
                                field_columns=field_columns,
                                tag_columns=[])
            time.sleep(bwait)
        time.sleep(3)
        print('Done!')
    else:
        print('Write-out not enabled. Skipping...')

DateLaZi

In [40]:
#new method from DateLaZi
# dlz=requests.get('https://datelazi.ro/latestData.json').content
# dlz=requests.get('https://di5ds1eotmbx1.cloudfront.net/latestData.json').content
dlz=requests.get('https://d35p9e4fm9h3wo.cloudfront.net/latestData.json').content
dlz=json.loads(dlz)
dlz['historicalData'][dlz['currentDayStats']['parsedOnString']]=dlz['currentDayStats']
dlzs=[]
dlz_counties=[]
for date in dlz['historicalData']:
    d={}
    mdate=date.replace('2018-11-07','2020-11-07').replace('2020-01-05','2021-01-05').replace('2020-01-07','2021-01-07')
    d['date']=mdate
    d['cases']=dlz['historicalData'][date]['numberInfected']
    d['heals']=dlz['historicalData'][date]['numberCured']
    d['deaths']=dlz['historicalData'][date]['numberDeceased']
    if 'vaccines' in dlz['historicalData'][date]:
        if dlz['historicalData'][date]['vaccines']:
            for v in dlz['historicalData'][date]['vaccines']:
                for k in ['total_administered','immunized']:
                    k2=k.replace('immunized','total_immunized')
                    if k2 not in d: d[k2]=0
                    d[k2]+=dlz['historicalData'][date]['vaccines'][v][k]
                    d[k2+'_'+v]=dlz['historicalData'][date]['vaccines'][v][k]
    if 'countyInfectionsNumbers' in dlz['historicalData'][date]:
        counties=dlz['historicalData'][date]['countyInfectionsNumbers']
        if counties:
            for county in counties:
                dummy={'date':mdate,county:dlz['historicalData'][date]['countyInfectionsNumbers'][county]}
                if county=='PH':
                    if date=='2020-12-01':
                        dummy[county]=19395
                dlz_counties.append(dummy)
    dlzs.append(d)
dl=pd.DataFrame(dlzs).set_index('date').sort_index(ascending=True)

In [41]:
for c in dl.columns[3:]:
    dl[c]=dl[c].cumsum()
dl2=dl.diff()
dl2.columns=['case','heal','death']+[i.replace('total_','') for i in dl2.columns[3:]]
dl['active']=dl['cases']-dl['heals']-dl['deaths']
dl=dl.join(dl2)

In [42]:
latest=dl.index[-1]
now=str(pd.to_datetime("now"))[:10]
print(latest,now)

2021-05-02 2021-05-02


Tests from `graphs.ro`

In [43]:
r=requests.get('https://www.graphs.ro/json.php')
graphs_ro=json.loads(r.content)['covid_romania']
g=pd.DataFrame(graphs_ro)
# g['date']=pd.to_datetime(g['reporting_date'])
g['date']=g['reporting_date']
g=g.set_index('date')
g['tests']=g['total_tests']
g['test']=g['new_tests_today']
latest2=g.index[0]
print(latest2)

2021-05-02


In [44]:
latest=str(min(pd.to_datetime(latest),pd.to_datetime(latest2)))[:10]
print(latest)

2021-05-02


In [45]:
dl=dl.join(g[['tests','test']])

Vaccines from `graphs.ro`

_deprecated_

In [46]:
# r=requests.get('https://www.graphs.ro/vaccinare_json.php')#?data_date=2021-01-05
# vc=json.loads(r.content)

In [47]:
# vc=pd.DataFrame(vc['covid_romania_vaccination']).set_index('reporting_date')
# vc=vc[['total_first_dose']]
# vc.columns=['vaccines']
# # vc.index=pd.to_datetime(vc.index)
# vc.index.name='date'
# latest3=vc.index[0]
# print(latest3)

In [48]:
# latest=str(min(pd.to_datetime(latest),pd.to_datetime(latest3)))[:10]
# print(latest)

In [49]:
# dl=dl.join(vc)

Governance

In [50]:
sheet='governance'

In [51]:
df=pd.read_csv(url+sheet)
# dl=pd.concat([df.set_index('date').sort_index()[:'2020-03-17'][dl.columns[:-1]],
#           dl['2020-03-18':]])
df=df.set_index('date').sort_index(ascending='True').join(dl,lsuffix='_g')#[20:] #[:-1] vaccines nincs a governance-ban

In [52]:
# fill from local, if missing in DateLaZi
for c in ['active','cases','heals','deaths','case','heal','death']:
    df[c]=df[c].fillna(df[c+'_g'].astype(str).str.replace(',','')).replace('nan',np.nan)
df=df.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [53]:
df14=df[['cases']].reset_index().join(df[['cases']][14:].reset_index(),lsuffix='1')
df14['case14']=df14['cases'].astype(float)-df14['cases1'].astype(float)
df=df.join(df14.set_index('date')['case14'].dropna().astype(int))

In [54]:
df.index=pd.to_datetime(df.index)
df0=df.copy()
#df['date']=pd.to_datetime(df['date'])
#df=df[df.columns[:13]].set_index('date')
vaccine_totals=[i for i in list(dl2.columns[3:])+list(dl.columns[3:len(dl2.columns)]) if 'total_' in i]
df=pd.DataFrame(df[['active','cases','heals','deaths','case14']+\
                  vaccine_totals].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float).astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
current=int(df[df['type']=='case14']['value'][-1]) #sajat becsles
# current=df[df['type']=='active']['value'][-1] #official
current

28351

In [55]:
df0.to_csv('df0.csv')

In [56]:
#sanity check
cols=[ 'active','case14', 'case', 'heal', 'death','cases', 'heals', 'deaths']
dx=df0[cols].dropna(how='all')
dx=dx.loc[~dx.index.isnull()]
dx[cols].tail()

,active,case14,case,heal,death,cases,heals,deaths
date,,,,,,,,
2021-04-28,40906.0,35330.0,2240.0,3595.0,150.0,1051779.0,983040.0,27833.0
2021-04-29,39831.0,33328.0,1850.0,2787.0,138.0,1053629.0,985827.0,27971.0
2021-04-30,37763.0,31700.0,1636.0,3566.0,138.0,1055265.0,989393.0,28109.0
2021-05-01,35773.0,29533.0,1307.0,3212.0,85.0,1056572.0,992605.0,28194.0
2021-05-02,33867.0,28351.0,1083.0,2901.0,88.0,1057655.0,995506.0,28282.0


In [57]:
df1=df0[['cases', 'heals', 'deaths', 'total_administered',
   'total_administered_pfizer', 'total_immunized',
   'total_immunized_pfizer', 'total_administered_moderna',
   'total_immunized_moderna', 'total_administered_astra_zeneca',
   'total_immunized_astra_zeneca', 'active', 'case', 'heal', 'death',
   'administered', 'administered_pfizer', 'immunized', 'immunized_pfizer',
   'administered_moderna', 'immunized_moderna',
   'administered_astra_zeneca', 'immunized_astra_zeneca', 'tests', 'test',
   'case14']]

In [58]:
for c in df1:
    df1[c]=df1[c].astype(str).str.replace(',','').astype(float)
df1.to_csv('github/data/time_series_ro_daily.csv')

<ipython-input-58-096a9c4a3f22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[c]=df1[c].astype(str).str.replace(',','').astype(float)


-> Run till here to test Grafana only.

In [59]:
df1=df.replace(0,np.nan).dropna()
df1=df1[df1['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14'])]
df2=df.replace(0,np.nan).dropna()
df2=df2[~(df2['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14']))]

In [60]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance1'
# push2influx(df1,measurement,field_columns,tag_columns,fo=True)
push2influx(df1,measurement,field_columns,tag_columns)
measurement='vaccine1'
# push2influx(df2,measurement,field_columns,tag_columns,fo=True)
push2influx(df2,measurement,field_columns,tag_columns)

Slicing governance1 from 2021-05-03 00:00:00+00:00 ...
Writing to governance1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!
Slicing vaccine1 from 2021-05-03 00:00:00+00:00 ...
Writing to vaccine1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [61]:
# df=pd.read_csv(url+sheet)
df=df0.copy()

# df['date']=pd.to_datetime(df['date'])
# df=df[df.columns[:13]].set_index('date')

Daily cases

In [62]:
df['case/test']=100*df['case'].astype(str).str.replace(',','').astype(float)/\
    df['test'].astype(str).str.replace(',','').astype(float)
df['death_rate']=100*df['death'].astype(str).str.replace(',','').astype(float)/\
    df['case'].astype(str).str.replace(',','').astype(float)

In [63]:
df=pd.DataFrame(df[['death','heal','case','case/test','death_rate']].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']

In [64]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance2'
# push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns,fo=True)
push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns)

Slicing governance2 from 2021-05-03 00:00:00+00:00 ...
Writing to governance2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Forecast

In [65]:
sheet='C19RFMC'
df=pd.read_csv(url+sheet).drop('Day',axis=1)
df['date']=pd.to_datetime(df['Date'])
df=df.drop('Date',axis=1).set_index('date')
# latestw='2020-11-08'
latestw=df[['Low 95 line']].dropna().index[0]
df.loc[latestw+pd.to_timedelta('-1d')]=df.loc[latestw+pd.to_timedelta('-1d')].bfill().ffill()
df1=df.loc[:latestw+pd.to_timedelta('-1d')][['Cases (fact & forecast)']]
df1.columns=['M1']
df2=df.loc[latestw+pd.to_timedelta('-1d'):][['Cases (fact & forecast)']]
df2.columns=['M2']

In [66]:
df=df.join(df1).join(df2)
df=df.stack().reset_index().set_index('date')#.loc[latest:]
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').str.replace('%','').astype(float).astype(int)
df=df[~(df['type'].isin(['Cases (fact & forecast)', 'Low 80 line','High 80 line',]))]

In [67]:
df['type']=df['type'].replace({'M1':'Fact', 'Low 95 line':'-95%', 'Low 60 line':'-60%', 
                               'High 60 line':'+60%', 'High 95 line':'+95%',
       'M2':'Forecast'})

In [68]:
field_columns=['value']
tag_columns=['type']
measurement='forecast'
push2influx(df,measurement,field_columns,tag_columns,fo=True)

Purging forecast ...
Writing to forecast ...
527 data points will be written in 0.1054 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Global

In [69]:
sheet='countries'

In [70]:
df=pd.read_csv(url+sheet)
df['date']=pd.to_datetime(df['date'])
df=df[df.columns[:13]].set_index('date')
df=df.stack().reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').str.replace('%','').astype(float).astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']

In [71]:
# field_columns=['value']
# tag_columns=['type','lang','langtype']
# measurement='global1'
# push2influx(df,measurement,field_columns,tag_columns)

In [72]:
sheet='global' #annotations
df=pd.read_csv(url+sheet)

In [73]:
ds=pd.DataFrame(szotardf.stack())
ds.columns=['country']
ds=ds.reset_index()
ds['index2']=ds['ID']+ds['level_1']
dc=pd.DataFrame(df.set_index(['Dátum','tag1','Link'])[languages].stack()).reset_index()
dc['index1']=dc['tag1']+dc['level_3']
dc=dc.set_index('index1').join(ds.set_index('index2')).set_index('Dátum')[[0,'country','Link','level_1']]
dc.columns=['desc','country','link','lang']
dc.index=pd.to_datetime(dc.index)

In [74]:
field_columns=['desc','country','link']
tag_columns=['lang']
measurement='global2'
push2influx(dc,measurement,field_columns,tag_columns)

Slicing global2 from 2021-02-07 00:00:00+00:00 ...
Writing to global2 ...
3 data points will be written in 0.0006 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Global automatic

In [75]:
# countries=['Brazil','France','Germany','Italy','Romania','Spain','US']
# columns=['br','fr','de','it','ro','es','us']
countries=['Brazil','France','Germany','Italy','Romania','US','United Kingdom']
columns=['br','fr','de','it','ro','us','gb']
eus=['Austria','Belgium','Bulgaria','Croatia','Cyprus', 'Czechia', 'Denmark','Estonia',
    'Finland', 'France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg',
    'Malta','Netherlands','Poland', 'Portugal', 'Romania','Slovakia','Slovenia','Spain','Sweden']
eus+=['United Kingdom','Andorra','Armenia','Azerbaijan','Belarus','Bosnia and Herzegovina','Georgia', 
     'Holy See','Iceland','Liechtenstein', 'Moldova','Montenegro','North Macedonia',
       'Norway','Russia','San Marino','Switzerland','Turkey','Ukraine','Kosovo']

In [76]:
csvs=['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',
'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv']

In [77]:
dfs=[]
x=4
#sheets=['countries_auto_cases','countries_auto_heals','countries_auto_deaths']
for i in range(len(csvs)):
    csv=csvs[i]
    print(csv)
    df=pd.read_csv(csv)
    df.to_csv('data'+csv[csv.rfind('/'):])
    df=df[[c for c in df.columns if 'Unnamed' not in c]]
    ww=pd.DataFrame(df.T[x:].astype(float).sum(axis=1))
    ww.columns=['ww']
    cn=pd.DataFrame(pd.DataFrame(df[df['Country/Region']=='China']).T[x:].astype(float).sum(axis=1))
    cn.columns=['cn']
    eu=pd.DataFrame(pd.DataFrame(df[df['Country/Region'].isin(eus)]).T[x:].astype(float).sum(axis=1))
    eu.columns=['eu']
    df=df[df['Province/State'].astype(str)=='nan']
    df=df[df['Country/Region'].isin(countries)]
    df=df.set_index("Country/Region").T[x-1:]
    df=df[countries]
    df.columns=columns
    df=df.join(eu).join(cn).join(ww)
    df.index=pd.to_datetime(df.index)
    dfs.append(df)

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv


In [78]:
df=dfs[0]-dfs[1]-dfs[2]
df=df.drop('ro',axis=1)
df=df.stack().reset_index()
df.columns=['date','type','value']
df=df.set_index('date')
df['value']=df['value'].astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
df.index=pd.to_datetime(df.index)

In [79]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='global3'
push2influx(df,measurement,field_columns,tag_columns)

Slicing global3 from 2021-04-25 00:00:00+00:00 ...
Writing to global3 ...
189 data points will be written in 0.0378 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Global aligned

In [80]:
n=30
start=dfs[0][dfs[0]['ro']>30].index.min()
print(start)

2020-03-11 00:00:00


In [81]:
df=dfs[0].rolling(7).mean().diff()
das=[]
for c in df:
    da=pd.DataFrame(df[c][df[c]>n])
    da.index=range(len(da))
    das.append(da)
das=pd.concat(das,axis=1)
das.index=[pd.to_datetime(start)+pd.to_timedelta('1D')*i for i in range(len(das))]
df=das.stack().reset_index()
df.columns=['date','type','value']
df=df.set_index('date')
df['value']=df['value'].astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
df.index=pd.to_datetime(df.index)

In [82]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='global4'
push2influx(df,measurement,field_columns,tag_columns)

Slicing global4 from 2021-06-06 00:00:00+00:00 ...
Writing to global4 ...
21 data points will be written in 0.0042 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Social

In [83]:
sheet='governance'
df=pd.read_csv(url+sheet)
df['date']=pd.to_datetime(df['date'])
df=df.set_index('date')[df.columns[10:22]].dropna(how='all')

In [84]:
c='gov_note_econ'
dc=df[[c,'gov_note_econ_tag','law','law_link']].dropna(subset=[c]).reset_index()\
            .set_index(c).join(szotardf[languages]).reset_index()
dc['date']=resolve_time_conflicts(dc['date'])
dc=pd.DataFrame(dc.set_index(list(dc.columns[:-3])).stack()).reset_index()
dc['index']=dc['law']+dc['level_5']
dc=dc.set_index('index')
dc.columns=['note','date','tag','law','link','lang','desc']
ds=szotardf.stack().reset_index()
ds['index']=ds['ID']+ds['level_1']
ds=ds.set_index('index')
ds.columns=['id','lang2','desc2']
dc=dc.join(ds).set_index('date')[['desc','tag','link','lang','desc2']]
dc['index']=dc['tag']+dc['lang']
ds.columns=['id','lang3','desc3']
dc=dc.reset_index().set_index('index').join(ds)
dc=dc.set_index('date')[['desc','link','lang','desc2','desc3']]

In [85]:
field_columns=['desc','desc2','link','desc3']
tag_columns=['lang']
measurement='social1'
push2influx(dc,measurement,field_columns,tag_columns,shift=False)
dc.to_csv('github/data/economic_measures_ro.csv')

Slicing social1 from 2020-04-27 00:00:00+00:00 ...
Writing to social1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [86]:
c='gov_note_social'
dc=df[[c,'gov_note_social_tag','law','law_link']].dropna(subset=[c]).reset_index()\
            .set_index(c).join(szotardf[languages]).reset_index()
dc['date']=resolve_time_conflicts(dc['date'])
dc=pd.DataFrame(dc.set_index(list(dc.columns[:-3])).stack()).reset_index()
dc['index']=dc['law']+dc['level_5']
dc=dc.set_index('index')
dc.columns=['note','date','tag','law','link','lang','desc']
ds=szotardf.stack().reset_index()
ds['index']=ds['ID']+ds['level_1']
ds=ds.set_index('index')
ds.columns=['id','lang2','desc2']
dc=dc.join(ds).set_index('date')[['desc','tag','link','lang','desc2']]
dc['index']=dc['tag']+dc['lang']
ds.columns=['id','lang3','desc3']
dc=dc.reset_index().set_index('index').join(ds)
dc=dc.set_index('date')[['desc','link','lang','desc2','desc3']]

In [87]:
field_columns=['desc','desc2','link','desc3']
tag_columns=['lang']
measurement='social2'
push2influx(dc,measurement,field_columns,tag_columns,shift=False)
dc.to_csv('github/data/social_measures_ro.csv')

Slicing social2 from 2021-04-08 00:00:00+00:00 ...
Writing to social2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [88]:
c='gov_note_fin'
dc=df[[c,'fin_tag','fin_link','fin_body']].dropna(subset=[c]).reset_index()\
            .set_index(c).join(szotardf[languages]).reset_index()
dc['date']=resolve_time_conflicts(dc['date'])
dc=pd.DataFrame(dc.set_index(list(dc.columns[:-3])).stack()).reset_index()
dc['index']=dc['fin_body']+dc['level_5']
dc=dc.set_index('index')
dc.columns=['note','date','tag','link','law','lang','desc']
ds=szotardf.stack().reset_index()
ds['index']=ds['ID']+ds['level_1']
ds=ds.set_index('index')
ds.columns=['id','lang2','desc2']
dc=dc.join(ds).set_index('date')[['desc','tag','link','lang','desc2']]
dc['index']=dc['tag']+dc['lang']
ds.columns=['id','lang3','desc3']
dc=dc.reset_index().set_index('index').join(ds)
dc=dc.set_index('date')[['desc','link','lang','desc2','desc3']]
dc['desc']=dc['desc'].str.replace('"','')

In [89]:
field_columns=['desc','desc2','link','desc3']
tag_columns=['lang']
measurement='social3'
push2influx(dc,measurement,field_columns,tag_columns,shift=False)
dc.to_csv('github/data/financial_measures_ro.csv')

Slicing social3 from 2021-03-16 00:00:00+00:00 ...
Writing to social3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Stocks

In [90]:
# sheet='stocks'
# df=pd.read_csv(url+sheet)[:-2].drop('Roman allampapir spreadek',axis=1)
# sheet='stocks2008'
# df2=pd.read_csv(url+sheet)[:-2]

In [91]:
# dc=pd.DataFrame(df.set_index(['Dátum']).stack()).reset_index()
# dc.columns=['date','stock','value']
# dc['year']=2020
# dc2=pd.DataFrame(df2.set_index(['Dátum','Dátum 2']).stack()).reset_index()
# dc2.columns=['date','date2','stock','value']
# dc2['year']=2008
# dc3=pd.concat([dc,dc2])
# dc3=pd.DataFrame(dc3.set_index('stock').join(szotardf).reset_index().set_index(['index','year','date','date2','value']).stack()).reset_index()
# dc3=dc3.set_index('date')
# dc3.columns=['stock','year','date2','value','lang','langstock']
# dc3['value']=dc3['value'].astype(str).str.replace(',','').str.replace('%','').astype(float)
# dc3.index=pd.to_datetime(dc3.index)

In [92]:
# field_columns=['value']
# tag_columns=['lang','langstock','year','date2','stock']
# measurement='stocks1'
# push2influx(dc3,measurement,field_columns,tag_columns)

In [93]:
sheet='stocks_all'
df=pd.read_csv(url+sheet)

In [94]:
stocks0=['S&P 500','DAX','BET','STOXX 600','FTSE 100']
stocksv0=['SP500 volatilitas','DAX volatilitas','BET volatilitas','Stoxx 600 volatilitas','FTSE 100 volatilitas']
stocksa0=['WTI','Brent']
stocksb0=['ROBOR3M']
stocksc0=['Roman allampapir spreadek','Bid/ask 6 honap','Bid/ask 12 honap ','Bid/ask 3 ev','Bid/ask 5 ev','Bid/ask 10 ev']
bets0=['BET','BET hozam']

In [95]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [96]:
v=[]
for s in stocks0:
    v.append(process.extractOne(s,list(df.columns))[0])
stocks=v
v=[]
for s in stocksv0:
    v.append(process.extractOne(s,list(df.columns))[0])
stocksv=v
v=[]
for s in stocksa0:
    v.append(process.extractOne(s,list(df.columns))[0])
stocksa=v
v=[]
for s in stocksb0:
    v.append(process.extractOne(s,list(df.columns))[0])
stocksb=v
v=[]
for s in stocksc0:
    v.append(process.extractOne(s,list(df.columns))[0])
stocksc=v
v=[]
for s in bets0:
    v.append(process.extractOne(s,list(df.columns))[0])
bets=v

In [97]:
open('bets.json','w').write(json.dumps(bets))

22

In [98]:
# bets=json.loads(open('bets.json','r').read())

In [99]:
das=[]
dbs=[]
for stock in stocks+stocksa:
    da=df[[stock,list(df.columns)[list(df.columns).index(stock)+1]]].dropna()
    da=da.set_index(stock)
    da.columns=[stock]
    da.index=pd.to_datetime(da.index)
    das.append(da)
for i,stock in enumerate(stocks):
    da=df[[stock,stocksv[i]]].dropna()
    da=da.set_index(stock)
    da.columns=[stocksv[i]]
    da.index=pd.to_datetime(da.index)
    das.append(da)
for d in das:
    dbs.append(d[~d.index.duplicated(keep='last')])
das=pd.concat(dbs,axis=1)
for stock in stocksb:
    da=df[[stock,list(df.columns)[list(df.columns).index(stock)+1]]].dropna()
    da=da.set_index(stock)
    da.columns=[stock]
    da.index=pd.to_datetime(da.index,dayfirst=True)
das=das.join(da)
da=df[stocksc].set_index(stocksc[0]).dropna()*100
da.index=pd.to_datetime(da.index,dayfirst=True)
das=das.join(da)

In [100]:
for c in das:
    das[c]=das[c].astype(str).str.replace(',','').str.replace('%','').astype(float)
das.to_csv('github/data/stocks.csv')

In [101]:
da1=das.sort_index(ascending=False)[:'2020-02-26']
da2=das.sort_index(ascending=False)[:'2008-09-16'].sort_index(ascending=False).tail(len(da1))
da2.index=da1.index
del das #free up mem
dc=pd.DataFrame(da1.stack()).reset_index()
dc.columns=['date','stock','value']
dc['year']=2020
dc2=pd.DataFrame(da2.stack()).reset_index()
dc2.columns=['date','stock','value']
dc2['year']=2008
del da1 #free up mem
del da2 #free up mem
dc3=pd.concat([dc,dc2])
del dc
del dc2
dc3['stock']=dc3['stock'].replace({(stocks+stocksv+stocksa+stocksb+stocksc)[i]:(stocks0+stocksv0+stocksa0+stocksb0+stocksc0)[i] for i in range(len(stocks+stocksv+stocksa+stocksb+stocksc))})
dc3=pd.DataFrame(dc3.set_index('stock').join(szotardf).reset_index().set_index(['index','year','date','value']).stack()).reset_index()
dc3=dc3.set_index('date')
dc3.columns=['stock','year','value','lang','langstock']
dc3['value']=dc3['value'].astype(str).str.replace(',','').str.replace('%','').astype(float)
dc3.index=pd.to_datetime(dc3.index)

In [102]:
field_columns=['value']
tag_columns=['lang','langstock','year','stock']
measurement='stocks1'
push2influx(dc3,measurement,field_columns,tag_columns)

Slicing stocks1 from 2021-04-17 00:00:00+00:00 ...
Writing to stocks1 ...
1230 data points will be written in 0.246 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [103]:
sheet='exchangerates'
df=pd.read_csv(url+sheet)[:-1]

In [104]:
for c in df:
    df[c]=df[c].astype(str).str.replace(',','').str.replace('%','')
df.to_csv('github/data/exchange_rates.csv')

In [105]:
dc=pd.DataFrame(df.set_index(['date']).stack()).reset_index()
dc.columns=['date','stock','value']
dc=pd.DataFrame(dc.set_index('stock').join(szotardf)\
.reset_index().set_index(['index','date','value']).stack()).reset_index()
dc=dc.set_index('date')
dc.columns=['stock','value','lang','langstock']
dc['value']=dc['value'].astype(str).str.replace(',','').str.replace('%','').astype(float)
dc.index=pd.to_datetime(dc.index)

In [106]:
field_columns=['value']
tag_columns=['lang','langstock','stock']
measurement='stocks2'
push2influx(dc,measurement,field_columns,tag_columns)

Slicing stocks2 from 2021-04-24 00:00:00+00:00 ...
Writing to stocks2 ...
24 data points will be written in 0.0048 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Firms

In [107]:
sheet='firms'
df=pd.read_csv(url+sheet)

In [108]:
dc=df[['Friendly name','Üzleti forgalom, 2018 (RON)','Alkalmazottak száma, 2018 (fő)',
       'Részarány az országos üzleti forgalomból, 2018 (%)','Részarány az iparági üzleti forgalomból, 2018 (%)']]
dc.columns=['name','revenue','employees','share','indshare']
dc['date']=pd.to_datetime('2020-04-05')
dc=dc.set_index('date').dropna(how='all')
dc['employees']=dc['employees'].astype(str).str.replace(',','').astype(float).astype(int)
dc['revenue']=dc['revenue'].astype(str).str.replace(',','').astype(float).astype(int)
dc['share']=dc['share'].astype(str).str.replace('%','').astype(float)
dc['indshare']=dc['indshare'].astype(str).str.replace('%','').astype(float)

<ipython-input-108-ed307922f6be>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc['date']=pd.to_datetime('2020-04-05')


In [109]:
field_columns=['revenue','employees','share','indshare']
tag_columns=['name']
measurement='firms1'
push2influx(dc,measurement,field_columns,tag_columns)

Slicing firms1 from 2020-04-06 00:00:00+00:00 ...
Writing to firms1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Firms2 at page bottom

In [110]:
dc2=df[['Friendly name','Higiénia, távolságtartás', 'Informálás', 'Adomány',
       'Üzleti utak leállítása vagy korlátozása',
       'Home office, rugalmas munkaidő', 'Részleges leállás', 'Teljes leállás',
       'Személyzet csökkentése (akár somai tehnic-el)',
       'Támogatás (pl. telefonos vonal stb.)', 'Testhőmérséklet követése',
       'Egyéb',
       'Súlyosság (4 = nagyon súlyos, 3 = súlyos, 2 = átlagos, 1 = gyenge)']].set_index('Friendly name')
dc3=dc.reset_index().set_index('name').join(dc2).reset_index().set_index('date')
dc3.columns=['name','revenue','employees','share','indshare','higenia',
'informalas',
'adomany',
'uzleti_utak',
'home_office',
'reszleges_leallas',
'teljes_leallas',
'szemelyzet_csok',
'tamogatas',
'testho',
'egyebi',
'sulyos']

In [111]:
field_columns=list(dc3.columns[1:])
tag_columns=['name']
measurement='firms3'
push2influx(dc3.fillna(0),measurement,field_columns,tag_columns)

Slicing firms3 from 2020-04-06 00:00:00+00:00 ...
Writing to firms3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [112]:
dc2=df[['Friendly name','COVID-19 terjedésének megakadályozása','Alkalmazottak egészségének védelme','Ügyfelek egészségének védelme','Csökkent kereslet','Támogatás (egészségügyi, kapcsolattartás)',
        'A vállalat lépéseinek indoka(i) HU','A vállalat lépéseinek indoka(i) RO','A vállalat lépéseinek indoka(i) EN',
       'Súlyosság (4 = nagyon súlyos, 3 = súlyos, 2 = átlagos, 1 = gyenge)']].set_index('Friendly name')
dc3=dc.reset_index().set_index('name').join(dc2).reset_index().set_index('date')
dc3.columns=['name','revenue','employees','share','indshare','terjedes',
'alkegeszseg',
'ugyfegeszseg',
'csokkereslet',
'tamogatasek',
'HU','RO','EN',
'sulyos']

In [113]:
dc3=dc3.reset_index().set_index(['date','name','revenue','employees','share','indshare','terjedes',
'alkegeszseg',
'ugyfegeszseg',
'csokkereslet',
'tamogatasek',
'sulyos']).stack().reset_index().set_index('date')
dc3.columns=['name','revenue','employees','share','indshare','terjedes',
'alkegeszseg',
'ugyfegeszseg',
'csokkereslet',
'tamogatasek',
'sulyos',
'lang',
'lepesindok']

In [114]:
field_columns=['revenue','employees','share','indshare','terjedes',
'alkegeszseg',
'ugyfegeszseg',
'csokkereslet',
'tamogatasek',
'sulyos',
'lepesindok']
tag_columns=['name','lang']
measurement='firms4'
push2influx(dc3.fillna(0),measurement,field_columns,tag_columns)

Slicing firms4 from 2020-04-06 00:00:00+00:00 ...
Writing to firms4 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Feeds

In [115]:
# feeds=['https://stirioficiale.ro/feeds/video.xml',
#       'https://cors-anywhere.herokuapp.com/https://stirioficiale.ro/feeds/hotarari.xml',
#       'https://cors-anywhere.herokuapp.com/https://stirioficiale.ro/feeds/informatii.xml',
#       'https://www.graphs.ro/seo-rss.rss']

In [116]:
r=requests.get('http://rssmix.com/u/12642387/rss.xml')
open(htmlipath+'panels/rss.xml','wb').write(r.content)

1010427

News

In [117]:
sheet='sajto'
df=pd.read_csv(url+sheet)

In [118]:
df=df.set_index('Datum')
df.index=pd.to_datetime(df.index)
df.columns=['type','media','desc','lang','link']

In [119]:
# Make sure datum is unique
ds={}
ts=[]
for i in df.T.iteritems():
    d=str(i[0])
    if d not in ds:
        ds[d]=pd.to_datetime(d)
        t=(ds[d])
    else:
        ds[d]=ds[d]+pd.to_timedelta('1m')
        t=(ds[d])
    ts.append(t)
df['Datum1']=ts
df=df.set_index('Datum1')

In [120]:
news_icons={
    'video':'🎥', 
    'news':'📰', 
    'talk':'🎤'
}
df['type']=df['type'].replace(news_icons)
df['desc']=df['desc'].str.replace('\"','\`')
df=df.drop_duplicates().sort_index(ascending=False)

In [121]:
field_columns=['type','desc','lang','link','media']
tag_columns=[]
measurement='news1'
push2influx(df,measurement,field_columns,tag_columns,fo=True)
df.to_csv('github/data/media.csv')

Purging news1 ...
Writing to news1 ...
391 data points will be written in 0.0782 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Oxford indicator

In [122]:
iso3_2={'BRA':'br','FRA':'fr','DEU':'de','GBR':'gb','ITA':'it','ROU':'ro','USA':'us'}

In [123]:
# # deprecated 2021-02-02
# stringency=requests.get('https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-02-26/'+now).content
# stringency=json.loads(stringency)

In [124]:
# stringdata=[]
# for date in stringecy['data']:
#     for country in stringecy['data'][date]:
#         if country in iso3_2:
#             stringdata.append({'date':date,'iso3':country,
#                                'stringency':stringecy['data'][date][country]['stringency'],'cases':stringecy['data'][date][country]['confirmed']})
# df=pd.DataFrame(stringdata)
# df=df.set_index(['date','iso3']).stack().reset_index()

In [125]:
stringency_csv=pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv')
stringency_csv=stringency_csv[((stringency_csv['CountryCode'].isin(iso3_2))&\
                               (stringency_csv['Jurisdiction']=='NAT_TOTAL'))]
stringency_csv.index=pd.to_datetime(stringency_csv['Date'].astype(str).str[:4]+'-'+\
    stringency_csv['Date'].astype(str).str[4:6]+'-'+\
    stringency_csv['Date'].astype(str).str[6:])
stringency_csv=stringency_csv[['CountryCode','StringencyIndexForDisplay','ConfirmedCases']].loc['2020-02-26':]
stringency_csv.columns=['country','stringency','cases']
df=stringency_csv.set_index('country',append=True).stack().reset_index()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [126]:
stringency_csv.to_csv('github/data/oxford_stringency.csv')

In [127]:
df.columns=['date','iso3','type','value']
df=df.join(pd.DataFrame(iso3_2,index=['country']).T,on='iso3').join(szotardf[languages],on='country')\
    .drop(['iso3'],axis=1).set_index(['date','value','type','country']).stack().reset_index()\
    .join(szotardf[languages],on='type')
df.columns=['date','value','type','lang','country','langcountry']+languages
df=df.set_index(['date','value','type','lang','country','langcountry']).stack().reset_index()
df.columns=['date','value','type','country','lang','langcountry','lang2','langtype']
df=df.set_index('date')
df.index=pd.to_datetime(df.index)
df['value']=df['value'].astype(float)
severity=df[df['country']=='ro']

In [128]:
severity.to_csv('severity.csv')

In [129]:
# severity=pd.read_csv('severity.csv').set_index('date')

In [130]:
field_columns=['value']
tag_columns=['lang','country','lang2','type','langcountry','langtype']
measurement='severity'
push2influx(df,measurement,field_columns,tag_columns)

Slicing severity from 2021-04-26 00:00:00+00:00 ...
Writing to severity ...
819 data points will be written in 0.1638 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Memory check

In [131]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
def memo():
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
                             key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
memo()

                      mobility: 133.7 MiB
                            df: 20.7 MiB
                      severity:  3.0 MiB
                           df0: 570.2 KiB
                             g: 398.8 KiB
                      szotardf: 393.1 KiB
                stringency_csv: 248.1 KiB
                            da: 170.0 KiB
                  dlz_counties: 137.3 KiB
                            dl: 123.6 KiB


In [132]:
# del _60
# del _66
# del _67
# del _69
del df
del mobility
# del datagov4

Google mobility

Mobility counties - restart `InfluxDB` before proceeding with this step

In [133]:
import numpy as np

In [134]:
sheet='industry_county'
dc=pd.read_csv(url+sheet)[:-2]
dc=dc[dc.columns[:3]].dropna()
dc.columns=['HU','RO','EN']
dc.loc['ro']=szotardf.loc['ro'][['HU','RO','EN']]

In [135]:
measurement='mobility'

In [136]:
#try migratin to dask
google_mobility_url='https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
field_columns=['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']
tag_columns=['sub_region_1']
filter_columns=['country_region_code']
time_columns=['date']
df_type={i:'Int64' for i in field_columns}
df_type={i:str for i in time_columns}#'M8[s]'
df_type.update({i:str for i in tag_columns+filter_columns})
iter_csv = pd.read_csv(google_mobility_url, usecols=field_columns+tag_columns+filter_columns+time_columns, 
                       dtype=df_type, iterator=True, chunksize=1000)
# latest=client.query('SELECT * FROM '+measurement+' GROUP BY * ORDER BY DESC LIMIT 1')
# lat=latest[list(latest.keys())[0]].index[0]
# df = pd.concat([chunk[((chunk['country_region_code'] == 'RO')&(pd.to_datetime(chunk['date'])>lat.replace(tzinfo=None)))] for chunk in iter_csv])
df = pd.concat([chunk[(chunk['country_region_code'] == 'RO')] for chunk in iter_csv])

In [137]:
df=df.dropna(subset=field_columns) #temporary workaround to remove all rows with at least one NaN, in order to counteract Google changing the reporting style #https://www.google.com/covid19/mobility/

In [138]:
df.to_csv('github/data/google_mobility_original.csv')

In [139]:
df=df.set_index('date')
df.index=pd.to_datetime(df.index)
df['sub_region_1']=df['sub_region_1'].str.replace(' County','')
field_columns=['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']
tag_columns=['sub_region_1']
df=df[field_columns+tag_columns]
df['sub_region_1']=df['sub_region_1'].fillna(dc.loc['ro']['RO'])

In [140]:
# #RESCALED BY FEB 15 - MAR 1
df=df.reset_index()
dki=df.set_index(['date','sub_region_1'])
# dkm=df.set_index('date').[:'2020-03-01'].groupby(['sub_region_1']).mean()+100
# dkm=(df.groupby(['date','sub_region_1']).mean()+100).loc[:'2020-03-01']
dkm=df.set_index('date').loc[:'2020-03-01'].groupby(['sub_region_1']).mean()+100
df=(dki/dkm).reset_index().set_index('date')

df=pd.DataFrame(df.set_index('sub_region_1',append=True).stack()).reset_index()
df['sub_region_1']=df['sub_region_1'].replace('Bucharest','București')
df=df.set_index('sub_region_1').join(dc.set_index('RO',drop=False))
df.columns=['date','indicator','value','HU','RO','EN']
mobility=pd.DataFrame(df.set_index(['date','indicator','value']).stack()).reset_index().set_index('date')
mobility.columns=['indicator','value','lang','langtype']
mobility['value']=np.round(mobility['value']*100,0)

In [141]:
mobility.to_csv('mobility.csv')

In [142]:
(dki/dkm).reset_index().set_index('date').to_csv('github/data/google_mobility_rescaled.csv')

In [143]:
# mobility=pd.read_csv('mobility.csv').set_index('date')

In [144]:
field_columns=['value']
tag_columns=['indicator','lang','langtype']
push2influx(mobility,measurement,field_columns,tag_columns)

Slicing mobility from 2021-04-22 00:00:00+00:00 ...
Writing to mobility ...
9900 data points will be written in 1.98 batches.
Expected query running time is: 7 seconds.
Writing batch 1 ...
Writing batch 2 ...
Done!


Mobility dimensions - restart `InfluxDB` before proceeding with this step

In [145]:
ds=pd.DataFrame(szotardf[['HU','RO','EN']].stack())
ds.index.names=['indicator','lang']
df=mobility.reset_index().set_index(['indicator','lang']).join(ds).reset_index().set_index('date')
df.columns=['indicator','lang','value','langtype','langindicator']
df=df[df['langtype'].isin(szotardf.loc['ro'][['HU','RO','EN']].values)][['value','lang','langindicator']]

In [146]:
field_columns=['value']
tag_columns=['lang','langindicator']
measurement='mobility2'
push2influx(df,measurement,field_columns,tag_columns)

Slicing mobility2 from 2021-04-22 00:00:00+00:00 ...
Writing to mobility2 ...
126 data points will be written in 0.0252 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Border crossings

In [147]:
sheet='bordercrossings'
db=pd.read_csv(url+sheet)
db['date']=pd.to_datetime(db['Dátum'])
db=db.set_index('date')[['Változás ']]
db.columns=['border']
db['border']=db['border'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index()

Border crossings - automated

In [148]:
# !pip install lxml

In [149]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [150]:
pol='https://www.politiadefrontiera.ro/ro/main/n-date-deschise-17/?s=compact&page_no='
sources={}

In [151]:
for page in range(0,1): #(0,40)
    print(page)
    urlp=pol+str(page)
    r = requests.get(urlp)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    s=soup.find('div',{'class':'news_grid_home'}).find_all('div',{'class':'article-compact'})
    dates=[i.find('time')['datetime'] for i in s]
    links=[i.find('a')['href'] for i in s]
    for i,link in enumerate(links):
        date=dates[i]
        if date not in sources:
            r = requests.get(link)
            html_content = r.text
            soup = BeautifulSoup(html_content, 'lxml')
            sources[date]=soup

0


In [152]:
for date in sources:
    open(htmlipath+'panels/daily/sources'+date+'.txt','w').write(json.dumps(str(sources[date])))

In [153]:
list(sources.keys())[0],list(sources.keys())[-1]

('2021-05-01', '2021-04-08')

In [154]:
from os import listdir
from os.path import isfile, join
mypath=htmlipath+'panels/daily/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
sources={i[7:17]:json.loads(open(mypath+i,'r' ).read()) for i in onlyfiles if 'sources' in i}

In [155]:
frontier={}

In [156]:
for date in sources:
    soup=BeautifulSoup(sources[date], 'lxml')
    if 'sfârșit de săptămână' in soup.find('title').text:
        stop=True
    elif 'ultimele 48 de ore' in soup.find('title').text:
        stop=True
    elif '24-25 decembrie' in soup.find('title').text:
        stop=True
    elif 'ultima săptămână' in soup.find('title').text:
        stop=True
    elif 'ultima săptămână' in soup.find('meta')['content']:
        stop=True
    elif 'ultimele 48 de ore' in soup.find('meta')['content']:
        stop=True
    elif 'sfârșit de săptămână' in soup.find('meta')['content']:
        stop=True
    else:
        stop=False
    if pd.to_datetime(date)>pd.to_datetime('2018-08-09'):
        if date=='2020-01-27': stop=True
        elif date=='2020-01-02': stop=True
        elif date=='2019-12-29': stop=True
        if not stop:
            
            if date not in frontier:
                try:
                    for j in range(7):
                        art=soup.find('div',{'class':'txtcontent'}).findAll('p')[j].text.replace(u'\xa0',' ')
                        art=art.replace(' peste ',' ')
                        if 'sensul de intrare' in art:
                            break
                except:
                    print('no j',date,soup.findAll('link')[1]['href'])
                    break
                date2=art[art.find('data')+4:art.find('data')+15].strip()
                vehicles=art[art.find('mijloace de transport')-8:art.find('mijloace de transport')].strip()
                
                #vehicles
                try:
                    vehicles=int(vehicles.replace('.','').replace('e ','')\
                                .replace('i ','').replace('u ',''))
                    if vehicles<10000:
                        print('vehicles 1',date,vehicles,soup.findAll('link')[1]['href'])
                except:
                    if date=='2020-05-29': vehicles=23600
                    elif date=='2020-05-03': vehicles=25900
                    elif date=='2020-04-20': vehicles=16700
                    elif date=='2020-04-18': vehicles=12600
                    elif date=='2019-08-25': vehicles=105100
                    elif date=='2018-08-09': vehicles=53000
                    elif date=='2018-08-08': vehicles=53040
                    else: 
                        print('vehicles 2',j, date,vehicles,soup.findAll('link')[1]['href'])
                        break
                
                #tir
                if date=='2021-03-21':
                    tir=11650
                elif pd.to_datetime(date)>pd.to_datetime('2020-05-15'):
                    tir=art[art.find('automarfare')-9:art.find('automarfare')].strip()
                    try:
                        tir=int(tir.replace('.','').replace('re ','')\
                                .replace('e ','').replace('v ','').replace(' de',''))
                        if tir<5000:
                            print('tir 1',date,tir,soup.findAll('link')[1]['href'])
                    except:
                        if date=='2021-01-09': tir=10000
                        else: 
                            print('tir 2',j, date,tir,soup.findAll('link')[1]['href'])
                            break
                else:
                    tir=np.nan
                
                #enter
                #enter=art[art.find('aproximativ ')+11:art.find('aproximativ ')+19].strip()
                enter=art[art.find('sensul de intrare au fost aproximativ ')+37:\
                        art.find('sensul de intrare au fost aproximativ ')+46].strip()
                try:
                    enter=int(enter.replace('.','').replace('re ','')\
                            .replace('e ','').replace('v ','').replace(' de','')\
                            .replace(' pe','').replace(' p',''))
                    if enter==0:
                        if date=='2021-01-09': enter=31700
                    elif enter==24:
                        enter=art[art.lower().find(' pe sensul de intrare ')-8:\
                                  art.lower().find(' pe sensul de intrare ')].strip()
                        enter=int(enter.replace('.','').replace('re ','')\
                            .replace('e ','').replace('v ','').replace(' de','')\
                            .replace(' pe','').replace(' p',''))
                    elif enter<10000:
                        print('enter 1',date,enter,soup.findAll('link')[1]['href'])
                except:
                    if date=='2019-08-25': enter=174700
                    elif date=='2019-02-06': enter=55100
                    elif date=='2019-12-22': enter=209000
                    elif date=='2019-12-21': enter=134500
                    elif date=='2018-12-23': enter=176500
                    elif date=='2018-12-22': enter=122200
                    elif date=='2018-04-02': enter=np.nan
                    elif date=='2018-12-14': enter=69300
                    elif date=='2018-10-16': enter=77600
                    else: 
                        enter=art[art.lower().find(' pe sensul de intrare ')-8:\
                                  art.lower().find(' pe sensul de intrare ')].strip()
                        try:
                            enter=int(enter.replace('.','').replace('re ','')\
                                .replace('e ','').replace('v ','').replace(' de','')\
                                .replace(' pe','').replace(' p',''))
                        except:
                            enter=art[art.find('sensul de intrare au fost ')+25:\
                                       art.find('sensul de intrare au fost ')+34].strip()
                            try:
                                enter=int(enter.replace('.','').replace('re ','')\
                                    .replace('e ','').replace('v ','').replace(' de','')\
                                    .replace(' pe','').replace(' p',''))
                            except:                                
                                print('enter 2',j, date,enter,soup.findAll('link')[1]['href'])
                                break
                
                #exit
                exit=art[art.find('ieşire')+6:art.find('ieşire')+14].strip()
                try:
                    exit=int(exit.replace('.','').replace('re ','')\
                            .replace('e ','').replace('v ','').replace(' de','')\
                            .replace(' p',''))
                    if exit==0:
                        if date=='2021-01-09': enter=44300
                    elif exit<10000:
                        print('exit 0',date,exit,soup.findAll('link')[1]['href'])
                except:
                    exit=art[art.lower().find('sensul de intrare şi')+20:\
                             art.lower().find('sensul de intrare şi')+28].strip()
                    try:
                        exit=int(exit.replace('.','').replace('re ','')\
                            .replace('e ','').replace('v ','').replace(' de','')\
                            .replace(' p',''))
                        if exit<10000:
                            print('exit 1', date,exit,soup.findAll('link')[1]['href'])
                    except:
                        if date=='2023-08-25': exit=174700
                        elif date=='2020-02-09': exit=156600
                        elif date=='2020-01-19': exit=186400
                        elif date=='2019-12-26': exit=np.nan
                        elif date=='2019-12-22': exit=105000
                        elif date=='2019-12-21': exit=90500
                        elif date=='2019-12-15': exit=158300
                        elif date=='2019-11-17': exit=174300
                        elif date=='2019-10-20': exit=610800
                        elif date=='2019-09-22': exit=219400
                        elif date=='2019-08-25': exit=np.nan
                        elif date=='2018-04-02': exit=np.nan
                        elif date=='2018-12-24': exit=83400
                        elif date=='2018-12-13': exit=59200
                        else: 
                            print('exit 2',j, date,exit,soup.findAll('link')[1]['href'],art)
                            break
                
                frontier[date]={'date':date,'date2':date2,
                                'url':soup.findAll('link')[1]['href'],
                                'vehicles':vehicles,'tir':tir,'enter':enter,'exit':exit}
        else:
            frontier[date]={}

tir 1 2021-01-09 0 https://www.politiadefrontiera.ro/ro/main/i-situatia-traficului-la-granita-de-vest-in-ultimele-24-de-ore-22224.html
tir 1 2021-01-03 4300 https://www.politiadefrontiera.ro/ro/main/i-situatia-traficului-la-frontiera-de-vest-in-ultimele-24-de-ore-22113.html
vehicles 1 2021-01-02 6100 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22112.html
tir 1 2021-01-02 2700 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22112.html
enter 1 2021-01-02 6300 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22112.html
exit 0 2021-01-02 7100 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22112.html
tir 1 2020-12-31 4700 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22099.html
tir 1 2020-12-29 4800 https://www.politiadefrontiera.ro/ro/main/i-situatia-la-frontiera-in-ultimele-24-de-ore-22

In [157]:
db2=pd.DataFrame(frontier).T
db2.index=pd.to_datetime(db2.index)
db2=db2.sort_index()
db2=db2.resample('1d').sum().replace(0,np.nan)#.interpolate(method='linear')
db2['net entry']=(db2['enter']-db2['exit'])

In [158]:
db2.dropna().to_csv('github/data/border_crossings.csv')

In [159]:
border_change={}
for i in db2.loc['2020-01-01':].T.iteritems():
    try:
        # ly=pd.to_datetime(str(int(str(i[0])[:4])-1)+str(i[0])[4:]) #tavalyi ev
        ly=pd.to_datetime('2019'+str(i[0])[4:]) #2019
        tm20=i[1]['enter']+i[1]['exit']
        tm19=db2.loc[ly]['enter']+db2.loc[ly]['exit']
        # tm20=i[1]['vehicles']
        # tm19=db2.loc[ly]['vehicles']
        change=(tm20-tm19)*100/tm19
    except:
        change=np.nan
    border_change[i[0]]=change
border_change=pd.DataFrame(border_change,index=['border_change']).T

In [160]:
db2=db2.join(border_change.join(db).bfill(axis=1)['border_change'])

In [161]:
db3=db2[['vehicles','tir','enter','exit','border_change','net entry']].stack().reset_index()\
    .set_index('level_0').dropna().join(szotardf[languages],on='level_1')
db3=pd.DataFrame(db3.set_index(['level_1',0],append=True).stack())
db3.columns=['a']
db3=db3.reset_index().set_index('level_0')
db3.columns=['type','value','lang','langtype']

In [162]:
field_columns=['value']
tag_columns=['lang','langtype','type']
measurement='mobility3'
# push2influx(db3.loc['2020-02-26':],measurement,field_columns,tag_columns,fo=True)
push2influx(db3.loc['2020-02-26':],measurement,field_columns,tag_columns)

Slicing mobility3 from 2021-04-26 00:00:00+00:00 ...
Writing to mobility3 ...
105 data points will be written in 0.021 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Mobility4 at page bottom

Mobility index

In [163]:
# import sys

# # These are the usual ipython objects, including this one you are creating
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:15]

memo()

                      mobility: 136.1 MiB
                      severity:  3.0 MiB
                           db3:  3.0 MiB
                           dki:  1.6 MiB
                            df:  1.3 MiB
                           df0: 570.2 KiB
                             g: 398.8 KiB
                      szotardf: 393.1 KiB
                           db2: 343.6 KiB
                            ds: 255.4 KiB


In [164]:
home=mobility.reset_index().set_index(['indicator','lang']).join(ds).reset_index().set_index('date')\
    .set_index(['indicator','lang']).loc['residential_percent_change_from_baseline'].loc['HU'][0].values[0]
dz=df[df['lang']=='HU'].set_index('langindicator',append=True)[['value']].unstack()['value']
dz[home]=-dz[home]
dz=pd.DataFrame(dz.T.mean())
dz.columns=['mobility_mini']
mobility_mini=dz.sort_index()

In [165]:
mobility_mini.to_csv('github/data/mobility_index.csv')

In [166]:
mobility_mini.to_csv('mobility_mini.csv')

In [167]:
# mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')

Company registry - industries

In [168]:
sheet='matrix'
df=pd.read_csv(url+sheet)[:-1]
df=df[['IND HU short','IND RO short','IND EN short','Industry CAEN']].set_index('Industry CAEN')
df.columns=languages

In [169]:
sheet='CegekCAEN'
db=pd.read_csv(url+sheet)

In [170]:
dd=db.set_index('CAEN kód').join(df)
dd=pd.DataFrame(dd.set_index(['Hónap', 'Típus','Aktív cégek száma összesen (2018)', 'Cégek száma (2020)',
       'Cégek száma (2019)', 'Változás az előző évhez képest (%)',
       'Aktív cégek számához arányosítva (%)']).stack()).reset_index().reset_index()
dd.columns=['index','Hónap', 'Típus','active2018', 'comp2020','comp2019', 'changeY','changeA','lang','sector']
ds=pd.DataFrame(szotardf[languages].stack())
ds.index.names=['Típus','lang']
ds.columns=['type']
dd=dd.set_index(['Típus','lang']).join(ds).reset_index()
for c in [ 'changeY','changeA']:
    dd[c]=(dd[c].astype(str).str.replace(',','').str.replace('%','').astype(float)/100.0).sort_index()

In [171]:
dd.index=pd.to_datetime('2020-'+(pd.to_datetime('2020-'+dd['Hónap'].astype(str)).dt.month+1).astype(str)+'-1')-pd.to_timedelta('1d')
dd.index.name='date'

In [173]:
field_columns=['active2018', 'comp2020','comp2019', 'changeY','changeA']
tag_columns=['lang','sector','type']
dd=dd[field_columns+tag_columns]
measurement='registry1'
push2influx(dd,measurement,field_columns,tag_columns)

Slicing registry1 from 2020-08-01 00:00:00+00:00 ...
Writing to registry1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [174]:
#save for Flourish
de=dd.reset_index().set_index(['lang','date','sector','type']).loc['RO']['changeY'].unstack().reset_index()
de['date']=de['date'].astype(str)
open(htmlipath+'panels/registry1.json','w').write(json.dumps(list(de.T.to_dict().values())))

23776

In [175]:
#save for Flourish
de=dd.reset_index().set_index(['lang','date','type','sector']).loc['RO']['changeY'].unstack().reset_index()
de['date']=de['date'].astype(str)
open(htmlipath+'panels/registry1b.json','w').write(json.dumps(list(de.T.to_dict().values())))

24189

In [176]:
#save for Flourish
de=dd.reset_index().set_index(['lang','sector','type','date']).loc['RO']['changeY'].reset_index()
de['date']=de['date'].astype(str)
de=de.set_index(['sector','type','date'])['changeY'].unstack().reset_index()
open(htmlipath+'panels/registry1c.json','w').write(json.dumps(list(de.T.to_dict().values())))

18070

Small dashboard

In [177]:
# mobility_mini=mobility_mini[:'2020-08-10']

In [178]:
sheet='minidashboard'
dm=pd.read_csv(url+sheet)
# sheet='governance'
# df=pd.read_csv(url+sheet)
# df['date']=pd.to_datetime(df['date'])
# df=df[df.columns[:10]].set_index('date')
# df=df0[list(dl.columns)+['tests','test']].dropna(how='all')
df=df0[list(dl.columns)].dropna(how='all')
df=df.loc[~df.index.isnull()]
for i in dm.set_index('ID')['UI'].iteritems():
    df[i[0]]=i[1]
sheet='EcMonitor'
du=pd.read_csv(url+sheet)
du=du.set_index('date').loc['Unemployment ratio'][2:].dropna()
du.index=[(pd.to_datetime('now')-pd.to_timedelta('11D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(du))][::-1]
du=pd.DataFrame(du)
du.columns=['unemployment']
du.index=pd.to_datetime(du.index)
df=df.join(du)
sheet='employmentdata'
de=pd.read_csv(url+sheet)
de['date']=pd.to_datetime(de['date'])
de=de[de.columns[:3]].set_index('date')
de.columns=['felbontott','felfuggesztett']
# df=df.join(de.ffill())
for c in de.columns:
    de[c]=de[c].astype(str).str.replace(',','').str.replace('%','').astype(float)#.interpolate(method='linear').dropna()
df=df.join(de)
df=df.join(mobility_mini)
for c in df.columns:
    df[c]=df[c].astype(str).str.replace(',','').str.replace('%','').astype(float)
sheet='exchangerates'
dx=pd.read_csv(url+sheet)
dx=dx[['date','EUR - RON (megváltozás)']][5:-1]
dx['date']=pd.to_datetime(dx['date'])
dx=dx.set_index('date')
dx.columns=['xch']
dx['xch']=dx['xch'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dx)
df['xch']=df['xch'].ffill()
sheet='stocks_all'
dk=pd.read_csv(url+sheet)[bets].dropna()
dk['date']=pd.to_datetime(dk[bets[0]])
dk=dk.set_index('date').drop(bets[0],axis=1)
dk.columns=['bet']
dk=dk.reindex(df.index)
dk['bet']=dk['bet'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dk)
sheet='MiniGDP'
dg=pd.read_csv(url+sheet)
dg1=dg[['Date','GDP (QoQ, SA)']].dropna()
dg1.index=[(pd.to_datetime('now')-pd.to_timedelta('2D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg1))][::-1]
dg2=dg[['Date.1','GDP']].dropna()
dg2.index=[(pd.to_datetime('now')-pd.to_timedelta('8D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg2))][::-1]
sheet='DFMChartYoY'
dg3=pd.read_csv(url+sheet).T
dg3=pd.read_csv(url+sheet,skiprows=19,nrows=2).T.iloc[1:].dropna()[1]
dg3.index=[(pd.to_datetime('now')-pd.to_timedelta('1D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg3))][::-1]
dg=dg1.join(dg2).join(dg3)[['GDP','GDP (QoQ, SA)',1]].astype(float)
dg.columns=['gdp','gdpq','employees']
dg.index=pd.to_datetime(dg.index)
df=df.join(dg)
df=df[:pd.to_datetime('now')]

In [179]:
field_columns=list(df.columns)
tag_columns=[]
measurement='governance3'
# push2influx(df,measurement,field_columns,tag_columns,wo=True,fo=True)
push2influx(df,measurement,field_columns,tag_columns,wo=True)

Slicing governance3 from 2021-05-03 00:00:00+00:00 ...
Writing to governance3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


# Grafana

In [ ]:
today0=now[-2:]
if today0[0]=='0': 
    today=today0[1]
else:
    today=today0
currents=str(current)[:-3]+'&nbsp;'+str(current)[-3:]
hour=str(pd.to_datetime('now')+pd.to_timedelta('3h'))[11:16]
str(szotar['report']['UI'][1:]),current,currents,today0,hour

In [ ]:
local_load=False #load from HU.json - useful when migrating

In [271]:
def darken2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0xfefefe >> 1))[2:]
def brighten2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0x7f7f7f << 1))[2:]
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.rgb2hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [ ]:
fa={'status':'bar-chart', 'global':'globe','governance':'gavel','stocks':'line-chart','firms':'car','data':'database',
    'industry':'industry','exports':'sign-out','credits':'coffee','others':'ellipsis-h','counties':'street-view','macro':'forward',
   'dijak':'trophy','mobility':'train','reference':'pencil','imobiliare':'home','news':'newspaper-o'}
ids={8:'logo',90:'flogo', 10:'title',6:'main_content',12:'status', 19:'global', 18:'governance', 17:'stocks', 16:'firms', 15:'industry', 
     14:'counties',121:'footer',57:'footer2',23:'lang',66:'credits',82:'report',85:'others',93:'update',94:'data',98:'macro',
    106:'dijak',108:'mobility',13:'reference',124:'imobiliare',144:'news'}
plots={21:'governance_plot1',24:'governance_plot2',134:'governance_plot3',22:'global_plot1',32:'social_plot1',37:'social_plot3',36:'social_plot5',
      55:'stock',54:'none',42:'stock_plot3',56:'stock_plot4',41:'stock',43:'stock',44:'stock',46:'stock',45:'stock',
      48:'stock',49:'stock',50:'stock',51:'stock',53:'stock',61:'firms3',63:'firms5',96:'stock_plot5',95:'stock_plot6',
      100:'macro_scatter',110:'mobility_plot1',111:'mobility_plot2',107:'residential_percent_change_from_baseline_plot',
      112:'workplaces_percent_change_from_baseline_plot',115:'transit_stations_percent_change_from_baseline_plot',
      117:'grocery_and_pharmacy_percent_change_from_baseline_plot',116:'parks_percent_change_from_baseline_plot',
      114:'retail_and_recreation_percent_change_from_baseline_plot',132:'imobiliare_plot1',133:'imobiliare_plot2',
       127:'imobiliare_plot3',126:'imobiliare_plot4',135:'none',136:'megye_plot1',
       137:'megye_plot2',152:'vaccine_plot1',
      139:'severity_plot1',140:'severity_plot3'}
singlestats={68:'status_plot1',69:'status_plot2',70:'status_plot3',71:'status_plot4',72:'status_plot5',73:'status_plot6',74:'status_plot7',
            75:'status_plot14',76:'status_plot9',77:'status_plot10',78:'status_plot11',79:'status_plot12',80:'status_plot13',88:'status_plot10',89:'matrix',
            91:'megyecase',99:'GDP growth vs. market and DFM estimates',104:'macro_table',141:'megye_plot3',
            109:'mobility_map',125:'imobiliare_map1',128:'imobiliare_map2',138:'severity_plot2',
            145:'news1',143:'news3'}
heatmaps={59:'firms1',60:'firms2'}
tables={31:'global_plot2',33:'social_plot2',39:'social_plot4',38:'social_plot6',62:'firms4',64:'firms6',65:'firms7',146:'news2'}
tooltips=[64,65]
legends={64:86,65:87}
legendtext={legends[i]:'' for i in tooltips}
strftimes={'HU':'%Y.%m.%d','RO':'%d/%m/%Y','EN':'%Y-%m-%d'}
themes={'dark':{'HU':'Sötét kinézet','RO':'Temă închisă','EN':'Dark theme'},'light':{'HU':'Világos kinézet','RO':'Temă deschisă','EN':'Light theme'}}
interactive={'HU':'teljes képernyő','RO':'ecran complet','EN':'fullscreen'}
kep={'HU':'kép mentése','RO':'salvează imagine','EN':'save image'}
save={'HU':'adatok letöltése','RO':'descarcă datele','EN':'download data'}
generate_links=False

In [ ]:
def get_model(model,theme):
    css=''
    texts=[]
    for panel in model['panels']:
        if 'type' in panel:
            panel['description']=''
            panel['links']=[]
            links=[]
            t=''
            if theme!='dark':
                t='-'+theme+'?theme='+theme
            if panel['type']=='text':
                texts.append(panel['id'])
                if panel['id'] in ids:
                    i=ids[panel['id']]
                    if i=='logo':
                        panel['content']='<a target="_blank" href="'+\
                        szotar['logo_link'][lang]+'"><img id="logo" class="logo" alt="'+\
                        szotar['logo_alt'][lang]+'" src="'+htmlepath+\
                        szotar['logo_img'][lang]+'" style="height:70px!important;width:200px!important;"/></a>'
                    elif i=='flogo':
                        panel['content']='<a target="_blank" href="'+\
                        szotar['flogo_link'][lang]+'"><img id="flogo" class="flogo" alt="'+\
                        szotar['flogo_alt'][lang]+'" src="'+htmlepath+\
                        szotar['flogo_img'][lang]+'" style="height:80px!important;width:330px!important;"/></a>'
                    elif i=='lang':
                        panel['content']='<br><div class="lang" style="text-align: center; font-size:16px;">'
                        for l in ['RO','HU','EN']:
                            panel['content']+='&nbsp;&nbsp;<a target="_self" title="'+titles[l]+'" href="/d/'+uids[l]+t+'">'+l+'</a>'
                        panel['content']+='&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<i class="fa fa-paint-brush"></i>&nbsp;&nbsp;<a target="_self" title="'+themes['dark'][lang]+'" href="/d/'+\
                            uids[lang]+'"><i class="fa fa-circle"></i></a>'+\
                            '&nbsp;&nbsp;<a target="_self" title="'+themes['light'][lang]+'" href="/d/'+uids[lang]+'-light?theme=light"><i class="fa fa-circle-o"></i></a>'
                        panel['content']+='&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a style="font-size:20px;" target="_blank" title="Facebook" '+\
                        'href="https://www.facebook.com/covid19.roeim"><i class="fa fa-facebook"></i></a>'
                        panel['content']+='&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a style="font-size:20px;" target="_blank" title="Help" '+\
                        'href="https://github.com/denesdata/covid19-romania/wiki"><i class="fa fa-question"></i></a>'
                        panel['content']+='</div>'
                    elif i=='title':
                        panel['content']='<div style="text-align:left;"><h1>'+str(szotar['title'][lang])+'</h1><h3> '+str(szotar['subtitle'][lang])+'</h3></div>'
                    elif i in ['main_content','footer','footer2']:
                        panel['content']=str(szotar[i][lang])
                    elif i in ['credits','others','data','dijak']:
                        panel['content']='<p><h4><i class="fa fa-'+fa[i]+'"></i>&nbsp;&nbsp;'+str(szotar[i][lang])+'</h4><div>'+str(szotar[i+'_content'][lang])+'</div></p>'
                    elif i in ['reference']:
                        panel['content']='<p><h4><i class="fa fa-'+fa[i]+'"></i>&nbsp;&nbsp;'+str(szotar[i][lang])+'</h4><div>'+str(szotar[i+'_content'][lang])+\
                        ' '+pd.to_datetime('now').strftime(strftimes[lang])+'</div></p>'
                    elif i=='report':
                        panel['content']=('<p><span style="font-size:18px;">'+str(szotar[i][lang])+'</span></p>')\
                            .replace('$','<span style="font-size:26px;color:'+model['panels'][8]['colors'][0]+';"><b>'+str(szotar[i]['UI'][1:])+'</b></span>')\
                            .replace('+','<span style="font-size:26px;color:'+model['panels'][8]['colors'][1]+';"><b>'+str(current)+'</b></span>')
                    elif i=='update':
                        panel['content']='<div style="text-align:right;"><i class="fa fa-clock-o"></i>&nbsp;'+str(szotar['update'][lang])+' <b>'+pd.to_datetime('now').strftime(strftimes[lang])+'</b></div>'
                    else:
                        panel['content']='<p><h2><i class="fa fa-'+fa[i]+'"></i>&nbsp;&nbsp;'+str(szotar[i][lang])+'</h2><div>'+str(szotar[i+'_content'][lang])+'</div></p>'
                elif panel['id'] in legendtext:
                    panel['content']=legendtext[panel['id']]
            elif panel['type']=='graph':
                if panel['id'] in plots:
                    i=plots[panel['id']]
                    if (('stock' in i) or (panel['id'] in [54])):
                        panel['timeFrom']= str(int(szotar['report']['UI'][1:])+1)+'d'
                        panel['hideTimeOverride']= True
                    if i=='stock':
                        if lang in ['HU','RO']:
                            description=str(szotarHU[panel['title']][lang+'_description'])
                            source=str(szotarHU[panel['title']][lang+'_source']) 
                            panel['title']=szotarHU[panel['title']][lang]
                        else:
                            description=str(szotarRO[panel['title']][lang+'_description'])
                            source=str(szotarRO[panel['title']][lang+'_source'])     
                            panel['title']=szotarRO[panel['title']][lang]  
                        if description not in ['nan','XXX']:
                            panel['description']=description
                        if source not in ['nan','XXX']:
                            panel['description']+='  \n📊 '+source
                    elif i!='none':
                        panel['title']=szotar[i][lang]
                        description=str(szotar[i][lang+'_description'])
                        source=str(szotar[i][lang+'_source'])    
                        if description not in ['nan','XXX']:
                            panel['description']=description
                        if source not in ['nan','XXX']:
                            panel['description']+='  \n📊 '+source
                    for s in panel['seriesOverrides']:
                        if 'alias' in s:
                            if s['alias'] not in ['/./']:
                                if lang in ['HU','RO']:
                                    s['alias']=szotarHU[s['alias']][lang]
                                else:
                                    s['alias']=szotarRO[s['alias']][lang]
                        if 'fillBelowTo' in s:
                            if lang in ['HU','RO']:
                                s['fillBelowTo']=szotarHU[s['fillBelowTo']][lang]
                            else:
                                s['fillBelowTo']=szotarRO[s['fillBelowTo']][lang]
                    for s in panel['targets']:
                        if 'alias' in s:
                            if lang in ['HU','RO']:
                                sz={s:szotarHU[s] for s in szotarHU if szotarHU[s]['UI']=='replace'}
                                if s['alias'] in sz:
                                    s['alias']=sz[s['alias']][lang]
                            else:
                                sz={s:szotarRO[s] for s in szotarRO if szotarRO[s]['UI']=='replace'}
                                if s['alias'] in sz:
                                    s['alias']=szotarRO[s['alias']][lang]

            elif panel['type'] in ['singlestat','ryantxu-ajax-panel','heatmap','news']:
                if panel['id'] in singlestats:
                    if (panel['id'] in [79,80]):
                        panel['timeFrom']= str(int(szotar['report']['UI'][1:])+1)+'d'
                        panel['hideTimeOverride']= True
                    i=singlestats[panel['id']]
                    panel['title']=szotar[i][lang]
                    description=str(szotar[i][lang+'_description'])
                    source=str(szotar[i][lang+'_source'])    
                    if description not in ['nan','XXX']:
                        panel['description']=description
                    if source not in ['nan','XXX']:
                        panel['description']+='  \n📊 '+source
                if panel['type'] in ['ryantxu-ajax-panel']:
                    panel['url']=panel['url'].replace(htmlepath_other,htmlepath)
            elif panel['type'] in ['savantly-heatmap-panel']:
                if panel['id'] in heatmaps:
                    i=heatmaps[panel['id']]
                    if lang in ['HU','RO']:
                        panel['title']=szotarHU[panel['title']][lang]
                    else:
                        panel['title']=szotarRO[panel['title']][lang]
                    description=str(szotar[i][lang+'_description'])
                    source=str(szotar[i][lang+'_source'])    
                    if description not in ['nan','XXX']:
                        panel['description']=description
                    if source not in ['nan','XXX']:
                        panel['description']+='  \n📊 '+source  
            elif panel['type']=='table-old':
                if panel['id'] in tables:
                    i=tables[panel['id']]
                    panel['title']=szotar[i][lang]
                    legend='<table class="legend"><tr><td>&nbsp;</td><td>&nbsp;</td></tr>'
                    for s in panel['styles']:
                        if 'alias' in s:
                            if lang in ['HU','RO']:
                                sz={s:szotarHU[s] for s in szotarHU if szotarHU[s]['UI']=='replace'}
                                if s['alias'] in sz:
                                    s['alias']=sz[s['alias']][lang]
                            else:
                                sz={s:szotarRO[s] for s in szotarRO if szotarRO[s]['UI']=='replace'}
                                if s['alias'] in sz:
                                    s['alias']=szotarRO[s['alias']][lang]
                        if panel['id'] in tooltips:
                            if 'link' in s:
                                if s['link']:
                                    s['linkTooltip']=szotar[s['pattern']][lang]
                                    legend+='<tr><td>'+s['alias']+'</td><td>'+szotar[s['pattern']][lang]+'</td></tr>'
                    if panel['id'] in tooltips:
                        legendtext[legends[panel['id']]]=legend
                    description=str(szotar[i][lang+'_description'])
                    source=str(szotar[i][lang+'_source'])    
                    if description not in ['nan','XXX']:
                        panel['description']=description
                    if source not in ['nan','XXX']:
                        panel['description']+='  \n📊 '+source
            if generate_links:
                if panel['type'] in ['graph','ryantxu-ajax-panel','savantly-heatmap-panel']:
                    links.append({'targetBlank': True,'title': '🕹 '+interactive[lang],
                          'url': '/d/'+uids[lang]+'/?var-lang=$lang&var-theme=$theme&theme=$theme'+\
                                  '&width=900&height=500&tz=Europe%2FBucharest&fullscreen&panelId='+str(panel['id'])+'&${__url_time_range}'})
                    links.append({'targetBlank': True,'title': '📷 '+kep[lang],
                          'url': '/render/d-solo/'+uids[lang]+'/?var-lang=$lang&var-theme=$theme&theme=$theme'+\
                                  '&width=900&height=500&tz=Europe%2FBucharest&panelId='+str(panel['id'])+'&${__url_time_range}'})
            panel['links']=links

    model['templating']['list'][0]['current']['text']=lang
    model['templating']['list'][0]['current']['value']=lang
    model['templating']['list'][1]['current']['text']=theme
    model['templating']['list'][1]['current']['value']=theme
    for option in model['templating']['list'][0]['options']:
        option['selected']=False
        if option['value']==lang:
            option['selected']=True
    if theme=='dark':
        model['title']=titles[lang]
        model['uid']=uids[lang]
        model['id']=iids[lang]
        open(lang+'.json','w').write(json.dumps(model))
    else:
        model['title']=titles[lang]+' Light'
        model['uid']=uids_light[lang]
        model['id']=iids_light[lang]
        open(lang+'-light.json','w').write(json.dumps(model))
    for text in texts:
        css+='\n#panel-'+str(text)+' .panel-menu-container {display: none !important;}\n#panel-'+\
        str(text)+' .panel-title-container {cursor: auto !important;}'
    for table in [107,112,115,117,116,114]:
        css+='\n#panel-'+str(table)+' .graph-legend-table .graph-legend-icon {display: none;}\n#panel-'+\
        str(table)+' .graph-legend-table td{padding: 1px 2px;}'
    for table in [107,112,115,117,116,114]:
        css+='\n#panel-'+str(table)+' .graph-legend-table thead {display: none;}\n#panel-'+\
        str(table)+' .graph-legend-table tr td:first-child{width:80px !important;display: grid;}'
    for table in [110,111]:
        css+='\n#panel-'+str(table)+' .graph-legend-table tr td:first-child{width:190px !important;display: inline-block;}'
    open(htmlipath+'/style/grafana/custom.'+theme+'.css','w').write(css)
    return model

In [ ]:
# szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

In [ ]:
# response = requests.get(grafana+'api/dashboards/uid/'+uids['HU'], headers=headers)

response = requests.get(grafana+'api/dashboards/uid/hu', headers=headers)

if local_load:
    model_dark=json.loads(open('HU.json','r').read().replace(htmlepath_other,htmlepath))
else:
    model_dark=json.loads(response.content)['dashboard']

# model_dark['time']['from']='now-'+str(szotar['report']['UI'][1:])+'d'
model_dark['time']['from']='now-'+str((pd.to_datetime('now')-pd.to_datetime('2020-03-13')).days)+'d'
model_dark['time']['to']='now+14d'
model_light=json.dumps(model_dark).replace('dark.css','light.css').replace('lightGray','#52545C')
colors_to_darken=['#F2CC0C','#CA95E5','#FF780A','#E0B400','#96D98D','#F2495C','#E02F44',
                  'lime','#73BF69','#8AB8FF','#3274D9','#D3D3D3','#FF7383','#FADE2A']
for color in colors_to_darken:
    model_light=model_light.replace(color,adjust_lightness(color,0.8))
    model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8))
model_light=json.loads(model_light)
models={'dark':model_dark,'light':model_light}
menu_sections={uids[i]:'' for i in languages}

In [ ]:
# for i in model_dark['panels']:
#     print(i['id'],i['type'],i['title'])

In [ ]:
def add_to_menus(model,lang):
    menu_sections[uids[lang]]='<table style="line-height:XRXRLHpx" class="XRXRCLASS"><tr>'+\
    '<td style="width: auto;" onclick="scroll_nicely(\'.dashboard-container\')">'+\
    '<div><i class="fa fa-arrow-circle-up"></i></div><div class="nav-text">'+str(szotar['start'][lang])+'</div></td>'
    for panel in model['panels']:
        if 'type' in panel:
            if panel['type']=='text':
                if panel['id'] in ids:
                    i=ids[panel['id']]
                    if 'content' in panel:
                        if (('<h2>' in panel['content']) or (i in ['credits'])):
                            m='<td onclick="scroll_nicely(\'#panel-'+str(panel['id'])+\
                            '\')"><div><i class="fa fa-'+fa[i]+'"></i></div><div class="nav-text">'+\
                                str(szotar[i][lang])+'</div></td>'
                            menu_sections[uids[lang]]+=m
    menu_sections[uids[lang]]+='</tr></table>'

In [ ]:
nevuto={'0':'á','1':'é','3':'á','2':'á','4':'é','5':'é','6':'á','7':'é','8':'á','9':'é'}
for lang in languages:
    response = requests.get(grafana+'api/dashboards/uid/'+lang.lower(), headers=headers)
    model=json.loads(response.content)['dashboard']
    for i in model['panels']:
        if lang=='HU':
            #print(i['id'],i['type'],i['title'])
            if i['id'] in (179, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find('n,')-4].strip()+' '+today+'-'+\
                    nevuto[now[-1]]+i['description'][i['description'].find('n,'):\
                    i['description'].find(':')-2]+hour+\
                    i['description'][i['description'].find(':')+3:]
                i['title']=i['title'][:i['title'].rfind(' ')]+' '+today
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='EN':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' on '+today+' '+ i['description'][i['description'].find('on ')+5:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='RO':
            if i['id'] in (153, 91):
                i['options']['iframeURL']=i['options']['iframeURL'][:i['options']['iframeURL'].find('.html#')-2]+\
                    today0+i['options']['iframeURL'][i['options']['iframeURL'].find('.html#'):]
                i['description']=i['description'][:i['description'].find(':')-2]+hour+' în data de '+today+' '+ i['description'][i['description'].find('în data de ')+13:]
                i['title']=i['title'][:i['title'].find('📆')+2]+today+i['title'][i['title'].find('📆')+3:]
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('<b>')+3]+today0+\
                i['content'][i['content'].rfind('<b>')+5:]
            
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })       
    print(r,r.content)
    open(lang+'.json','w').write(json.dumps(model))    
    
    model_light=json.dumps(model).replace('dark','light',9999).replace('lightGray','#52545C',9999)\
        .replace('#d3d3d3','#52545c',9999)\
        .replace('csaladen.es/favicon.ico" style="','csaladen.es/favicon.ico" style="filter: brightness(0.3);',9999)
    for color in colors_to_darken:
        model_light=model_light.replace(color,adjust_lightness(color,0.8),9999)
        model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8),9999)
    model=json.loads(model_light)
    model['title']=titles[lang]+' Light'
    model['uid']=uids_light[lang]
    model['id']=iids_light[lang]
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })    
    print(r,r.content)
    open(lang+'-light.json','w').write(json.dumps(model))   

In [193]:
#FIX LIGHT THEME CSS

In [ ]:
import re
dcss=open(htmlipath+'/style/grafana/dark.css','r').read()

In [ ]:
lcss=dcss.replace('#243b55','#abcaee',9999)\
.replace('lightGray','rgb(82, 84, 92)',9999)\
.replace('#141e30','#ecf2ff',9999)\
.replace('rgba(0, 0, 0, 0.6)','rgba(255, 255, 255, 0.6)',9999)\
.replace('#60c5eb','#86b2f6',9999)\
.replace('#d8d9da','#52545c',9999)\
.replace('#d3d3d3','#52545c',9999)\
.replace('#161719','#f7f8fa',9999)\
.replace('#fff','#1e2028',9999)\
.replace('#343436','#c7d0d9',9999)\
.replace('#8e8e8e','#767980',9999)\
.replace('#b3b3b3','#acb6bf',9999)\
.replace('#262628','#dde4ed',9999)\
.replace('#33b5e5','#5794f2',9999)\
.replace('#60c5eb','#86b2f6',9999)\
.replace('#ecbb13','#ff851b',9999)\
.replace('#0b0c0e','#fff',9999)\
.replace('#c7d0d9','#41444b',9999)\
.replace('#ececec','#41444b',9999)\
.replace('#97999c','#767980',9999)\
.replace('#1C2D43','#cedff4',9999)\
.replace('checkbox.png','checkbox_white.png',9999)\
.replace('#e9edf2','#141414',9999)\
.replace('icons_dark_theme','icons_light_theme',9999)\
.replace('#d8d9da','#f7f8fa',9999)\
.replace('#000','#52545c',9999)\
.replace('brightness(50)','brightness(1)',9999)\
.replace('dark','light',9999)+' body, a{color:#52545c !important;} .credit a, .panel-info-content a, .tooltip {color: lightGray !important;}'+\
    ' .flogo {filter: brightness(0.5);}'

In [ ]:
open(htmlipath+'/style/grafana/light.css','w').write(lcss)

In [ ]:
# for theme in ['dark','light']:
#     for lang in languages:
#         print(theme,lang)  
#         model=get_model(models[theme],theme)
#         if theme=='dark':
#             add_to_menus(model,lang)
#         r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
#                                                                         "folderId": folder_id,
#                                                                         "overwrite": True
#                                                                        })
#         print(r,r.content)

In [ ]:
# menu_sections

# Annotations

Restart `InfluxDB` and `Grafana` to proceed to free up memory

In [122]:
process_annoations=True

In [123]:
sleep_time=0.05

In [124]:
def purge_annotations(tag, lang):
    if process_annoations:
        for iid in [iids, iids_light]:
            time.sleep(1)
            dash=iid[lang]
            if tag=='all':
                r=requests.get(grafana+'api/annotations?limit=10000&dashboardId='+
                               str(dash), headers=headers)
            else:
                r=requests.get(grafana+'api/annotations?limit=10000&tags='+tag+
                               '&dashboardId='+str(dash), headers=headers)
            annotations=json.loads(r.content)
            print('Expected running time for query is:',int(len(annotations)*sleep_time*2),'seconds.')
            for a in annotations:
                time.sleep(sleep_time)
                r=requests.delete(grafana+'api/annotations/'+str(a['id']), headers=headers)
            print(lang,dash,len(annotations),'annotations purged')

In [125]:
def post_annotations(a,lang):
    if process_annoations:
        for iid in [iids, iids_light]:
            time.sleep(sleep_time)
            a["dashboardId"]=iid[lang]
            requests.post(grafana+'api/annotations', headers=headers, json=a)

In [126]:
forras={'HU':'forrás','RO':'sursa','EN':'source'}

In [127]:
if purge:
    for lang in ['HU','RO','EN']:
        purge_annotations('all',lang)

Global

In [128]:
sheet='global'
df=pd.read_csv(url+sheet)

In [129]:
df.to_csv('github/data/global_annotations.csv')

In [130]:
mtag='global'
for lang in ['HU','RO','EN']:
    purge_annotations(mtag,lang)
    for i in df.T.to_dict().values():
        tags=[mtag]
        if str(i['tag1'])!='nan':
            tags.append(szotar[str(i['tag1'])][lang])
        if str(i['tag2'])!='nan':
            tags.append(szotar[str(i['tag2'])][lang])
        a={
          "panelId":22,
          "time":int(pd.to_datetime(i['Dátum']).strftime("%s"))*1000,
          # "timeEnd":int((pd.to_datetime(i['Dátum'])+pd.to_timedelta('9h')).strftime("%s"))*1000,
          "tags":tags,
          "text":'<div class="annotation_"'+mtag+'>'+i[lang]+'</div><a target="_blank" href="'+i['Link']+'">'+forras[lang]+'</a>',
        }
        post_annotations(a,lang)
    print(lang)

Expected running time for query is: 6 seconds.
HU 2 63 annotations purged
Expected running time for query is: 6 seconds.
HU 5 63 annotations purged
HU
Expected running time for query is: 6 seconds.
RO 3 63 annotations purged
Expected running time for query is: 6 seconds.
RO 6 63 annotations purged
RO
Expected running time for query is: 6 seconds.
EN 4 63 annotations purged
Expected running time for query is: 6 seconds.
EN 7 63 annotations purged
EN


Governance - economic

In [131]:
sheet='governance'
df=pd.read_csv(url+sheet)
dc=df.dropna(subset=['gov_note_econ'])[['gov_note_econ','gov_note_econ_tag','date','law_link']]
dc['date']=resolve_time_conflicts(dc['date'])

In [132]:
mtag='economic'
for lang in ['HU','RO','EN']:
    purge_annotations(mtag, lang)
    for i in dc.T.to_dict().values():
        tags=[mtag]
        if str(i['gov_note_econ_tag'])!='nan':
            tags.append(szotar[str(i['gov_note_econ_tag'])][lang])
        a={
          "panelId":32,
          "time":int(pd.to_datetime(i['date']).strftime("%s"))*1000,
          # "timeEnd":int((pd.to_datetime(i['date'])+pd.to_timedelta('9h')).strftime("%s"))*1000,
          "tags":tags,
          "text":'<div class="annotation_"'+mtag+'>'+szotar[i['gov_note_econ']][lang]+\
            '</div><a target="_blank" href="'+i['law_link']+'">'+forras[lang]+'</a>',
        }
        post_annotations(a,lang)
    print(lang)

Expected running time for query is: 0 seconds.
HU 2 6 annotations purged
Expected running time for query is: 0 seconds.
HU 5 6 annotations purged
HU
Expected running time for query is: 0 seconds.
RO 3 6 annotations purged
Expected running time for query is: 0 seconds.
RO 6 6 annotations purged
RO
Expected running time for query is: 0 seconds.
EN 4 6 annotations purged
Expected running time for query is: 0 seconds.
EN 7 6 annotations purged
EN


Governance - social

In [133]:
sheet='governance'
df=pd.read_csv(url+sheet)
dc=df.dropna(subset=['gov_note_social'])[['gov_note_social','gov_note_social_tag','date','law_link']]
dc['date']=resolve_time_conflicts(dc['date'])

In [134]:
mtag='social'
for lang in ['HU','RO','EN']:
    purge_annotations(mtag, lang)
    for i in dc.T.to_dict().values():
        tags=[mtag]
        if str(i['gov_note_social_tag'])!='nan':
            tags.append(szotar[str(i['gov_note_social_tag'])][lang])
        a={
          "panelId":37,
          "time":int(pd.to_datetime(i['date']).strftime("%s"))*1000,
          # "timeEnd":int((pd.to_datetime(i['date'])+pd.to_timedelta('9h')).strftime("%s"))*1000,
          "tags":tags,
          "text":'<div class="annotation_"'+mtag+'>'+szotar[i['gov_note_social']][lang]+\
            '</div><a target="_blank" href="'+i['law_link']+'">'+forras[lang]+'</a>',
        }
        post_annotations(a,lang)
    print(lang)

Expected running time for query is: 2 seconds.
HU 2 24 annotations purged
Expected running time for query is: 2 seconds.
HU 5 24 annotations purged
HU
Expected running time for query is: 2 seconds.
RO 3 24 annotations purged
Expected running time for query is: 2 seconds.
RO 6 24 annotations purged
RO
Expected running time for query is: 2 seconds.
EN 4 24 annotations purged
Expected running time for query is: 2 seconds.
EN 7 24 annotations purged
EN


Governance - financial

In [135]:
sheet='governance'
df=pd.read_csv(url+sheet)
dc=df.dropna(subset=['gov_note_fin'])[['gov_note_fin','fin_tag','date','fin_body','fin_link']]
dc['date']=resolve_time_conflicts(dc['date'])

In [136]:
mtag='financial'
for lang in ['HU','RO','EN']:
    purge_annotations(mtag, lang)
    for i in dc.T.to_dict().values():
        tags=[mtag]
        if str(i['fin_body'])!='nan':
            tags.append(szotar[str(i['fin_body'])][lang])
        if str(i['fin_tag'])!='nan':
            tags.append(szotar[str(i['fin_tag'])][lang])
        a={
          "panelId":36,
          "time":int(pd.to_datetime(i['date']).strftime("%s"))*1000,
          # "timeEnd":int((pd.to_datetime(i['date'])+pd.to_timedelta('9h')).strftime("%s"))*1000,
          "tags":tags,
          "text":'<div class="annotation_"'+mtag+'>'+szotar[i['gov_note_fin']][lang]+\
            '</div><a target="_blank" href="'+i['fin_link']+'">'+forras[lang]+'</a>',
        }
        post_annotations(a,lang)
    print(lang)

Expected running time for query is: 1 seconds.
HU 2 10 annotations purged
Expected running time for query is: 1 seconds.
HU 5 10 annotations purged
HU
Expected running time for query is: 1 seconds.
RO 3 10 annotations purged
Expected running time for query is: 1 seconds.
RO 6 10 annotations purged
RO
Expected running time for query is: 1 seconds.
EN 4 10 annotations purged
Expected running time for query is: 1 seconds.
EN 7 10 annotations purged
EN


Firms

In [137]:
sheet='events'
df=pd.read_csv(url+sheet)
dc=df[['Friendly name','Date','HU', 'RO','EN','Link', 'Link.1', 'Link.2']]
dc.columns=['name','date','HU','RO','EN','link1','link2','link3']

In [138]:
sheet='firms'
df=pd.read_csv(url+sheet)
de=df[['Friendly name', 'RO CAEN sectiune','HU CAEN sectiune', 'EN CAEN sectiune']]
de.columns=['name','RO', 'HU','EN']
de=de.set_index('name').stack().reset_index()
de.columns=['name','lang','industry']

In [139]:
dc=dc.set_index(['name','date','link1','link2','link3']).stack().reset_index()
dc.columns=['name','date','link1','link2','link3','lang','steps']
dc=dc.set_index(['name','lang']).join(de.set_index(['name','lang'])).reset_index()

In [140]:
annos1=[]

In [141]:
if process_annoations:
    for lang in ['HU','RO','EN']:
        purge_annotations('firms', lang)
        purge_annotations('facebook', lang)
        dc3=dc[dc['lang']==lang]
        ds={}
        for i in dc3.T.to_dict().values():
            links=''
            links1=['']
            mtag='firms'
            pid=61
            atag=szotar['vallalati'][lang]
            for link in ['link1','link2','link3']:
                l=str(i[link])
                if l!='nan':
                    if links=='':
                        links='<a target="_blank" href="'+l+'">'+forras[lang]+'</a>'
                        links1[0]=l
                        if 'facebook' in l:
                            mtag='facebook'
                            pid=63
                            atag='Facebook'
                    else:
                        links+='; <a target="_blank" href="'+l+'">'+forras[lang]+'</a>'
                        links1.append(l)
            tags=[mtag]
            if str(i['name'])!='nan':
                tags.append(str(i['name']))
            if str(i['industry'])!='nan':
                tags.append(str(i['industry']))
            d=str(i['date'])
            if d=='nan':
                tags.append(str('no date'))
                d='2020-03-10'
            if d not in ds:
                ds[d]=pd.to_datetime(d)
                t=(ds[d])
            else:
                ds[d]=ds[d]+pd.to_timedelta('193m')
                t=(ds[d])
            a={
              "panelId":pid,
              "time":int(t.strftime("%s"))*1000,
              # "timeEnd":int((pd.to_datetime(t)+pd.to_timedelta('9h')).strftime("%s"))*1000,
              "tags":tags,
              "text":'<div><b>'+str(i['name'])+'</b></div><div class="annotation_"'+mtag+'>'+i['steps']+\
                '</div>'+links,
            }
            post_annotations(a,lang)
            annos1.append({'name':str(i['name']),'date':t,'event':i['steps'],'link':links1[0],'type':atag,'lang':lang})
        print(lang)

Expected running time for query is: 6 seconds.
HU 2 63 annotations purged
Expected running time for query is: 6 seconds.
HU 5 63 annotations purged
Expected running time for query is: 9 seconds.
HU 2 92 annotations purged
Expected running time for query is: 9 seconds.
HU 5 92 annotations purged
HU
Expected running time for query is: 6 seconds.
RO 3 63 annotations purged
Expected running time for query is: 6 seconds.
RO 6 63 annotations purged
Expected running time for query is: 9 seconds.
RO 3 92 annotations purged
Expected running time for query is: 9 seconds.
RO 6 92 annotations purged
RO
Expected running time for query is: 6 seconds.
EN 4 63 annotations purged
Expected running time for query is: 6 seconds.
EN 7 63 annotations purged
Expected running time for query is: 9 seconds.
EN 4 92 annotations purged
Expected running time for query is: 9 seconds.
EN 7 92 annotations purged
EN


Push annos to Influx

In [142]:
if process_annoations:
    da=pd.DataFrame(annos1).set_index('date')
    da['event']=da['event'].str.replace('"','')

In [143]:
if process_annoations:
    da3=[]
    for lang in ['HU','RO','EN']:
        da2=da[da['lang']==lang].sort_index(ascending=False)
        ds={}
        ts=[]
        for d in da2.index:
            if d not in ds:
                ds[d]=pd.to_datetime(d)
                t=(ds[d])
            else:
                ds[d]=ds[d]+pd.to_timedelta('193m')
                t=(ds[d])
            ts.append(t)
        da2.index=ts
        da3.append(da2)
    da3=pd.concat(da3)

In [144]:
da3.to_csv('github/data/firms_annotations.csv')

In [145]:
if process_annoations:
    field_columns=['name','event','link','type']
    tag_columns=['lang']
    measurement='firms2'
    push2influx(da3,measurement,field_columns,tag_columns,shift=False,wo=process_annoations)

Slicing firms2 from 2020-10-05 00:00:00+00:00 ...
Writing to firms2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


# Maps

Industry county map

In [64]:
top=3

In [65]:
# iso_counties=pd.read_html('https://en.wikipedia.org/wiki/Counties_of_Romania')[1]\
#     .set_index(['ISO code[note 3]'])['County'].to_dict()
# iso_counties['BN']='Bistrița-Năsăud'
# iso_counties['CL']='Călărași'
# open(htmlipath+'panels/iso_counties.json','w').write(json.dumps(iso_counties))
iso_counties=json.loads(open(htmlipath+'panels/iso_counties.json','r').read())

In [66]:
georo=json.loads(open(htmlipath+'panels/romania-counties.json','r').read())
georoco={i['properties']['NAME_1']:i['properties']['ID_1'] for i in georo['objects']['ROU_adm1']['geometries']}   
georoco['București']=georoco['Bucharest']

In [67]:
sheet='industry_county'
df=pd.read_csv(url+sheet)[:-2]
df=df[df.columns[:5]]
df.columns=['HU','RO','EN','q2','q3']
for q in ['q2','q3']:
    df[q]=df[q].str.replace('%','').astype(float)
df=df.set_index('RO').join(pd.DataFrame(georoco,index=['county']).T).reset_index().dropna()
df.columns=['RO','HU','EN','q2','q3','county']

In [68]:
df.to_csv('github/data/sectoral_effects_county.csv')

In [69]:
for lang in languages:
    dc=df[[lang,'q2','q3','county']]
    dc.columns=['county','q2','q3','id']
    for c in ['q2','q3']:
        dd=dc.set_index([c])[['county']].sort_index(ascending=False)
        dds=[]
        ddd=dd.iloc[:top]
        ddd[c+'_high']=True
        dds.append(ddd)
        ddd=dd.iloc[-top:]
        ddd[c+'_high']=False
        dds.append(ddd)
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['county']).join(dds.reset_index().set_index(['county'])\
                                         [[c+'_display',c+'_high']]).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    dcqs=[]
    for c in ['q2','q3']:
        dcq=dc.set_index([c,'county'])[[c+'_display',c+'_high']]
        dcq.index.names=['value','county']
        dcq['quarter']=c
        dcq=dcq.reset_index().set_index(['quarter','county','value'])
        dcq.columns=['display','high']
        dcq['high']=dcq['high'].fillna(False)
        dcqs.append(dcq)
    dcqs=pd.concat(dcqs)
    dcc=dc.set_index(['county','id'])[['q2','q3']].stack().reset_index()
    dcc.columns=['county','id','quarter','value']
    dcc=dcc.set_index(['quarter','county','value']).join(dcqs).reset_index()
    dcc['q']=dcc['quarter'].replace({'q2':'Q2 vs Q1','q3':'Q3 vs Q2'})
    dcc['q0']=dcc['quarter'].replace({'q2':2019,'q3':2020})
    # dcc=dcc.sort_values('quarter',ascending=False)
    open(htmlipath+'panels/county_'+lang+'.json','w').write(json.dumps(list(dcc.T.to_dict().values())))
    # open('county_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

<ipython-input-69-1a3a71a98fd4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=True
<ipython-input-69-1a3a71a98fd4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=False


Case county map

In [70]:
sheet='counties_current'
dg=pd.read_csv(url+sheet, header=None)
dgl=dg[[1,4]].set_index(4).T.to_dict()
dgl['Dambovita']=dgl['Dâmbovita']
dgl['Szucsáva']=dgl['Suceava']
dgl['Galac']=dgl['Galati']
dgl['Valcea']=dgl['Vâlcea']
dfl=df[languages+['county']].set_index('HU')
dh=dfl.join(pd.DataFrame(dgl).T).reset_index().dropna()

In [71]:
for lang in languages:
    dc=dh[[lang,1,'county']]
    dc.columns=['county','value','id']
    open(htmlipath+'panels/county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))
    # open('county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

Case county map fancy

In [72]:
pop=json.loads(open(htmlipath+'panels/pop.json').read())
pop={i:pop[i][1] for i in pop}
pdf=pd.DataFrame(pop,index=['pop']).T
pdf.index.name='iso'

In [73]:
dq=pd.DataFrame(dlz_counties).groupby('date').mean().drop('-',axis=1).stack().reset_index()
dq.columns=['date','iso','cases']
iso2=pd.DataFrame(iso_counties,index=['EN']).T.reset_index()
iso2.columns=['iso','EN']
dq=dq.set_index('iso').join(pdf).join(iso2.set_index('iso')).set_index('EN').join(dh.set_index('EN').drop(1,axis=1)).reset_index()
dq['case_cap']=dq['cases']*1000/dq['pop']

In [74]:
dmss=[]
for lang in languages:
    dms=[]
    # dm=dq.set_index([lang,'date'])[['cases','case_cap','pop','county']]
    dm=dq.set_index('EN').join(iso2.set_index('EN')).reset_index().set_index([lang,'date'])[['cases','case_cap','pop','county','iso']]
    for m in dm.index.get_level_values(0).unique():
        dc=dm.loc[m].sort_index()
        p=dc.iloc[0]['pop']
        dc['case_14']=dc['cases'].diff().rolling(14).sum()
        dc['case_14_cap']=dc['case_14']*1000/p
        dc['id']=dc['county']
        dc['county']=m
        dms.append(dc.dropna())
    dms=pd.concat(dms)
    dms=dms[((dms['case_14_cap']<14)&(dms['case_14_cap']>0))] #filter out non-sense values
    open(htmlipath+'panels/county2c_'+lang+'.json','w').write(json.dumps(list(dms.reset_index().T.to_dict().values())))
    dms2=dms.sort_index()['2020-08-05':]
    open(htmlipath+'panels/county2d_'+lang+'.json','w').write(json.dumps(list(dms2.reset_index().T.to_dict().values())))
    
    dc=dms.reset_index().sort_index(ascending=False)
    for c in ['cases','case_cap','case_14','case_14_cap']:
        dd=dc.reset_index().set_index(['date',c])[['county']].sort_index(ascending=False)
        dds=[]
        for d in dd.index.get_level_values(0).unique():
            dds.append(dd.loc[[d]].iloc[:top])
            dds.append(dd.loc[[d]].iloc[-top:])
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['date','county']).join(dds.reset_index().set_index(['date','county'])[c+'_display']).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    d=dc.iloc[[0]]
    d['county']='d'
    d['case_14_cap']=4
    d['id']=99
        
    open(htmlipath+'panels/county2b_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    open(htmlipath+'panels/daily/county2b_'+latest+'_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    dms['lang']=lang
    dmss.append(dms)
    print(lang)

<ipython-input-74-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-74-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-74-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

HU


<ipython-input-74-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-74-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-74-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

RO


<ipython-input-74-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-74-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-74-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

EN


In [75]:
dmss[1].to_csv('github/data/time_series_ro_counties_daily.csv')

In [76]:
cases_counties2=open(htmlipath+'panels/cases_counties2.html','r').read()
cases_counties2=cases_counties2[:cases_counties2.find('daily/county2b_\' + \'2021-')+20]+latest+cases_counties2[cases_counties2.find('daily/county2b_\' + \'2021-')+30:]
open(htmlipath+'panels/cases_counties2.html','w').write(cases_counties2)

18156

Counties per day

In [77]:
dz=dh.set_index('EN').join(pd.DataFrame(pd.DataFrame(iso_counties,index=['county'])\
                                     .T.reset_index().set_index('county'))).reset_index()
dlz_data=pd.DataFrame(pd.DataFrame(dlz_counties).groupby('date').mean().stack()).reset_index()
dlz_data.columns=['date','index','value']
dz=dlz_data.set_index('index').join(dz.set_index('index')).dropna().set_index('date')
dz.index=pd.to_datetime(dz.index)
dz2=dz.copy()

In [78]:
# dz2=dz2.ffill()

In [79]:
dz=pd.DataFrame(dz.set_index(['county','value',1],append=True).stack()).reset_index()
dz.columns=['date','county','value','dump','lang','langtype']
dz=dz.set_index('date').drop('dump',axis=1)

In [80]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties'
push2influx(dz,measurement,field_columns,tag_columns)

Slicing counties from 2021-05-03 00:00:00+00:00 ...
Writing to counties ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Counties active

In [81]:
dz2=dz2.reset_index().set_index(['EN','HU','RO','county',1,'date'])\
    .unstack().diff(axis=1).stack().reset_index().set_index('date')

In [82]:
dz2=pd.DataFrame(dz2.set_index(['county','value',1],append=True).stack()).reset_index()
dz2.columns=['date','county','value','dump','lang','langtype']
dz2=dz2.set_index('date').drop('dump',axis=1)

In [83]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties2'
push2influx(dz2,measurement,field_columns,tag_columns)

Slicing counties2 from 2021-05-03 00:00:00+00:00 ...
Writing to counties2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


County intensity map

In [84]:
dmsr=pd.concat(dmss)[['case_14_cap','county','iso','lang']]
dmsr.index=pd.to_datetime(dmsr.index)

In [85]:
field_columns=['case_14_cap']
tag_columns=['county','iso','lang']
measurement='counties3'
push2influx(dmsr,measurement,field_columns,tag_columns)

Slicing counties3 from 2021-05-03 00:00:00+00:00 ...
Writing to counties3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


County population density correlation

In [206]:
dpdc=dmsr.groupby('iso').max()

In [207]:
wiki_counties=pd.read_html('https://en.wikipedia.org/wiki/Counties_of_Romania')[1]

In [208]:
insse_pop=pd.read_csv(htmlipath+'panels/exportPivot_POP106A.csv')

In [209]:
wiki_counties['County2']=wiki_counties['County'].str.replace('ș','s').str.replace('ă','a')\
    .str.replace('ț','t').str.replace('â','a').str.replace('Bucharest','Municipiul Bucuresti')
insse_pop['County2']=insse_pop[' Macroregiuni  regiuni de dezvoltare si judete'].str.strip()

In [210]:
wiki_counties=wiki_counties.join(insse_pop.set_index('County2')[' Valoare'],on='County2')

In [211]:
# wiki_counties['pop_density']=wiki_counties['Population (2011)[21]']/\
#     wiki_counties['Area[22]'].astype(str).str.split('km2').str[0].str.replace(',','').astype(int)
wiki_counties['pop_density']=wiki_counties[' Valoare']/\
    wiki_counties['Area[22]'].astype(str).str.split('km2').str[0].str.replace(',','').astype(int)

In [212]:
dpdc.join(wiki_counties.set_index(['ISO code[note 3]'])).to_csv(htmlipath+'panels/case_density.csv')

In [213]:
dmsr['month']=dmsr.index.month
dpdc2=dmsr.groupby(['month','iso']).max().reset_index().set_index('iso')

In [214]:
dpdc2.join(wiki_counties.set_index(['ISO code[note 3]'])).to_csv(htmlipath+'panels/case_density2.csv')

Kepler export

In [ ]:
import copy

In [ ]:
roc=json.loads(open('../html/panels/ROU_adm1.json','r').read())

In [ ]:
roc_dict={i['properties']['ID_1']:i for i in roc['features']}

In [ ]:
dmsc=pd.concat(dmss)
dmsc=dmsc[dmsc['lang']=='RO'].reset_index()

In [ ]:
open('../html/panels/ROU_adm2.json','w').write(json.dumps(list(dmsc.T.to_dict().values())))

In [ ]:
rogeo=json.loads(open(htmlipath+'panels/ro.geo.json','r').read())
for f in rogeo['features']:
    f['properties']={'🌄':f['properties']['NAME_1']}
open(htmlipath+'panels/ro.simple.geo.json','w').write(json.dumps(rogeo))

Locality map

In [265]:
# datagov1a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/bc19c354-644d-4a24-a26f-512129dbc70d/download/transparenta_vaccinare_martie_2021.xlsx')
# datagov1b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/a0eb9ff2-9b97-430c-b285-360cadb55307/download/vaccinare-covid19-grupe-risc-01-01.04.2021.xlsx')

datagov2a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d0b60b45-fb08-4980-a34c-8cbb4a43cad3/download/transparenta_martie_2021.xlsx',skiprows=1)
datagov2b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/d3280256-07cc-4f93-957a-9815085899be/download/transparenta_aprilie_2021.xlsx',skiprows=1)

In [270]:
# datagov1=pd.concat([datagov1a,datagov1b])

datagov2=datagov2a.set_index(['Judet','UAT']).join(datagov2b.set_index(['Judet','UAT'])).reset_index()

In [271]:
# del datagov1a
# del datagov1b
del datagov2a
del datagov2b

In [272]:
uat=json.loads(open(htmlipath+'panels/uat_simplificat.geojson','r').read())

In [273]:
locs=[]
for f in uat['features']:
    locs.append(f['properties'])
locs=pd.DataFrame(locs)[['judet','uat','siruta']]

In [274]:
def judetconverter(c):
    return (' '.join([i.capitalize() for i in c.split(' ')])).replace('ţ','ț').replace('ş','ș').replace('năsăud','Năsăud').\
        replace('severin','Severin').replace('mare','Mare').\
        replace('Municipiul București','Bucharest').replace('București','Bucharest')

In [275]:
geo_iso=pd.DataFrame(iso_counties,index=['index']).T.reset_index().set_index('index').join(\
    pd.DataFrame(georoco,index=['geo_id']).T.reset_index().set_index('index')).reset_index()
geo_iso.columns=['en','iso2','geo_id']

In [276]:
locs['judet_norm']=[judetconverter(i) for i in locs['judet']]
datagov2['judet_norm']=[judetconverter(i) for i in datagov2['Judet']]

In [277]:
def uatconverter(judet,c):
    c=(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in c.split(' ')]))\
        .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
        .replace('î','â').replace(' De ',' de ')\
        .replace('Cuza Voda','Cuza Vodă')
    
    if (judet=='Dolj'):
        c=c.replace('Segarcea','Șegarcea')
    elif (judet=='Teleorman'):
        c=c.replace('Turnu Magurele','Turnu Măgurele')
    elif (judet=='Olt'):
        c=c.replace('Ipotesti','Ipotești')
    elif (judet=='Alba'):
        c=c.replace('Râmetea','Rimetea')
    elif (judet=='Constanța'):
        c=c.replace('44066','')
    elif (judet=='Ialomița'):
        c=c.replace('Radulești','Rădulești')
    elif (judet=='Argeș'):
        c=c.replace('Ciofringeni','Ciofrângeni')\
        .replace('Valea Mare Pravăț','Valea Mare-Pravăț')
    elif (judet=='Vâlcea'):
        c=c.replace('Păusești-Măglași','Păușești-Măglași')
    elif (judet=='Prahova'):
        c=c.replace('Cocorastii Colt','Cocorăștii Colț')
    elif (judet=='Bucharest'):
        c=c.replace('București Sectorul','Sector')
    elif (judet=='Gorj'):
        c=c.replace('Crușet','Crușeț')
    elif (judet=='Brăila'):
        c=c.replace('Racoviță','Racovița')\
        .replace('Gradiștea','Grădiștea')\
        .replace('Chișcani','Chiscani')
    elif (judet=='Cluj'):
        c=c.replace('Râșca','Rișca')
    elif (judet=='Botoșani'):
        c=c.replace('Răușeni','Răuseni')
    elif (judet=='Maramureș'):
        c=c.replace('Șisești','Șișești')
    elif (judet=='Brașov'):
        c=c.replace('Sambata de Sus','Sâmbăta de Sus')
    elif (judet=='Vaslui'):
        c=c.replace('Tătărăni','Tătărani')
    elif (judet=='Iași'):
        c=c.replace('Țigănăși','Țigănași')
    elif (judet=='Bistrița-Năsăud'):
        c=c.replace('Ilva Mica','Ilva Mică')
    elif (judet=='Bihor'):
        c=c.replace('Sănnicolau Romăn','Sânnicolau Român')
    elif (judet=='Mureș'):
        c=c.replace('Sarmașu','Sărmașu')
    return c

In [278]:
datagov2['uat_norm']=[(' '.join(['-'.join([b.capitalize() for b in a.split('-')]) for a in u.split(' ')]))\
                      .replace('ţ','ț').replace('ş','ș').replace('Ţ','Ț').replace('Ş','Ș')\
                      .replace(' De ',' de ').replace('Municipiul ','').replace('Oraș ','')\
                      for u in datagov2['UAT']]

In [279]:
uat_norms=[]
for judet in locs['judet_norm'].unique():
    for l in locs[locs['judet_norm']==judet]['uat'].unique():
        u=uatconverter(judet,str(l))
        if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
            u=u.replace(' ','-')
            if u not in datagov2[datagov2['judet_norm']==judet]['uat_norm'].unique():
                print(judet,l,u)
                u=''
        uat_norms.append({'judet_norm':judet,'uat_norm':u,'uat':l})

Constanța 44066 
  


In [280]:
locs2=locs.set_index(['judet_norm','uat']).join(pd.DataFrame(uat_norms).set_index(['judet_norm','uat'])).reset_index()

In [281]:
datagov3=datagov2.set_index(['judet_norm','uat_norm']).join(locs2.set_index(['judet_norm','uat_norm']))
datagov3=datagov3.set_index(['judet','uat','siruta'],append=True).drop(['UAT','Judet'],axis=1)
datagov3.index=datagov3.index.reorder_levels([2,3,4,0,1])
# datagov3_dates=[pd.to_datetime(c) for c in datagov3.columns]
# datagov3_dates=[c for c in datagov3.columns]
datagov3_dates=[str(pd.to_datetime(c)) for c in datagov3.columns]

In [282]:
datagov3=datagov3.round(2)

In [283]:
datagov3.reset_index().to_csv('github/data/time_series_ro_uat_daily.csv')

In [291]:
# del datagov1
del datagov2

In [292]:
def cat(r):
    if r<1: return '< 1 ‰'
    elif r<2: return '1-2 ‰'
    elif r<4: return '2-4 ‰'
    elif r<6: return '4-6 ‰'
    elif r<8: return '6-8 ‰'
    elif r<10: return '8-10 ‰'
    elif r<20: return '> 10-20 ‰'
    else: return '> 20 ‰'
def cat2(r):
    if r<1: return 0
    elif r<2: return 1
    elif r<4: return 2
    elif r<6: return 3
    elif r<8: return 4
    elif r<10: return 5
    elif r<20: return 6
    else: return 7

In [293]:
t=0
new_uat={'type':'FeatureCollection','features':[]}
new_uat_numbers=[]
new_uat_daily={'type':'FeatureCollection','features':[]}
for i in range(len(uat['features'])):
    dummy={}
    judet=uat['features'][i]['properties']['judet']
    if (judet!=''):
        u=uat['features'][i]['properties']['uat']
        siruta=uat['features'][i]['properties']['siruta']
        # dummy={'judet':judet,'uat':u,'siruta':siruta}
        if u in datagov3.loc[judet].index:
            d=datagov3.loc[judet].loc[u].loc[siruta]
            values=d.values[0]
            dummy['🌄']=d.index[0][0]
            dummy['🏠']=d.index[0][1]
            for t in range(len(values)):
                value=values[t]
                date=datagov3_dates[t]
                dummy['📈']=value
                dummy['🔴']=cat(value)
                dummy['⚫']=cat2(value)
                dummy['📆']=date
                feature={'type':'Feature','geometry':uat['features'][i]['geometry'],'properties':dummy.copy()}
                new_uat['features'].append(feature)
                new_uat_numbers.append(dummy.copy())
                if (t==len(datagov3_dates)-1):
                    new_uat_daily['features'].append(feature)

In [294]:
open(htmlipath+'panels/new_uat_numbers.json','w').write(json.dumps(new_uat_numbers))
open(htmlipath+'panels/new_uat.json','w').write(json.dumps(new_uat))

276910854

In [295]:
#Kepler.gl export
open(htmlipath+'panels/daily/new_uat'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/new_uat_daily.json','w').write(json.dumps(new_uat_daily))

7100456

In [296]:
del new_uat

In [297]:
#Mapbox colors:
# 239e69
# f5d232
# c52b69
# 850237

In [298]:
#D3plus export

In [299]:
datagov4=datagov3.stack().reset_index()
datagov4.columns=['judet','uat','id','judet_norm','uat_norm','date','value']
datagov4['value']=np.round(datagov4['value'],2)
# datagov4=datagov4.dropna(subset=['judet'])
datagov4=datagov4.dropna(how='any')

In [300]:
open(htmlipath+'panels/daily/uat2b_'+date[:10]+'.json','w').write(json.dumps(list(datagov4.T.to_dict().values())))

19389725

In [301]:
# #HTML overwrite
# cases_localities=open(htmlipath+'panels/cases_localities.html','r').read()
# cases_localities=cases_localities[:cases_localities.find('daily/uat2b_\' + \'2021-')+17]+latest+cases_localities[cases_localities.find('daily/uat2b_\' + \'2021-')+27:]
# open(htmlipath+'panels/cases_localities.html','w').write(cases_localities)
# #### CHECK IF LATEST DATE IS ONE LESS than latest

Restrictions

In [302]:
df2=pd.DataFrame(new_uat_numbers)
df4=df2[['🌄', '🏠', '📈','📆']].set_index(['🌄', '🏠', '📆']).unstack()['📈'].sort_index(axis=1)

In [303]:
def restrictie(x,w):
    if x>7.5: return 4
    elif x>7:
        if w>7.5: return 3
        else: return 2
    elif x>4: return 2
    elif x>3.5:
        if w>4: return 1
        else: return 0
    else:
        return 0
    
def restrictie2(x,w):
    if x>7.5: return '7.5+'
    elif x>7:
        if w>7.5: return '7.5-'
        else: return '4+'
    elif x>4: return '4+'
    elif x>3.5:
        if w>4: return '4-'
        else: return '0'
    else:
        return '0'

In [304]:
dr=[]
for i in df4.T.iteritems():
    v=i[1]
    #print(i[0])
    for j in range(len(v)):
        t=v.index[j]
        x=v[j]
        y=v[j-min(14,j):j].values
        w=x
        if len(y):
            w=max(y)
        r=restrictie(x,w)
        #print(j,t,x,w,r)
        dr.append({'🌄':i[0][0], '🏠':i[0][1], '🚦':r,'📆':t})

In [305]:
dp=pd.DataFrame(dr).set_index(['🌄', '🏠','📆']).astype(float)

In [306]:
for i in new_uat_daily['features']:
    i['properties']['🚦']=dp.loc[i['properties']['🌄']].loc[i['properties']['🏠']].loc[i['properties']['📆']]['🚦']

In [307]:
pd.DataFrame([i['properties'] for i in new_uat_daily['features']]).to_csv('time_series_ro_uat_restrictions_latest.csv')

In [308]:
open(htmlipath+'panels/new_uat_daily2.json','w').write(json.dumps(new_uat_daily))
open(htmlipath+'panels/daily/new_uat_daily2_'+date[:10]+'.json','w').write(json.dumps(new_uat_daily))

7167320

In [309]:
#HTML overwrite
uat_html=open(htmlipath+'panels/daily/mapbox-'+date[:10]+'.html','r').read()
# uat_html=uat_html[:uat_html.find('.json')-2]+date[8:10]+uat_html[uat_html.find('.json'):]
uat_html=uat_html[:uat_html.find('.json')-2]+\
    str(pd.to_datetime(now)+pd.to_timedelta('-1d'))[8:10]+\
    uat_html[uat_html.find('.json'):]
uat_html=uat_html.replace(date[5:10],date[5:8]+today0).replace(date[5:10],date[5:8]+today0)
open(htmlipath+'panels/daily/mapbox-'+date[:8]+today0+'.html','w').write(uat_html)

18658

Upload to `uat.json` to mapbox

In [ ]:
#Add szekelydata GitHub update
#Add mapbox direct upload

Mobility map

In [202]:
import numpy as np

In [203]:
dm=mobility[mobility['lang']=='HU']
dm=dm[dm['langtype']!=szotar['ro']['HU']]

# dn=dm.reset_index().set_index(['date','langtype','indicator'])[['value']].unstack()['value'].reset_index().set_index('date')
dm=dm.reset_index().set_index(['date','langtype','indicator'])[['value']]
dm=dm[~dm.index.duplicated(keep='first')]
dn=dm.unstack()['value'].reset_index().set_index('date')

dn['residential_percent_change_from_baseline']=-dn['residential_percent_change_from_baseline']
dw=dn.reset_index().set_index(['date']).sort_index()['2020-02-17':].groupby('langtype')\
    .resample('7D',label='right').mean().reset_index()
dw['date']-=pd.to_timedelta('1d')
dw['date']=dw['date'].dt.strftime('%Y-%b-%d').astype('str')#.str[:8]
dw=pd.DataFrame(np.round(dw.set_index(['date','langtype']).T.mean(),0)).reset_index()

In [204]:
# del mobility #free up memory
# del dm

In [205]:
dw=dw.join(dh.set_index('HU'),on='langtype').drop(1,axis=1)
dw.columns=['date','HU','mobility','RO','EN','county']
dw=dw.reset_index().dropna()
for lang in languages:
    dc=dw[['date',lang,'mobility','county']]
    dc.columns=['date','county','value','id']
    for c in ['value']:
        dd=dc.set_index(['date',c])[['county']].sort_index(ascending=False)
        dds=[]
        for d in dd.index.get_level_values(0).unique():
            dds.append(dd.loc[[d]].iloc[:top])
            dds.append(dd.loc[[d]].iloc[-top:])
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['date','county']).join(dds.reset_index().set_index(['date','county'])[c+'_display']).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        dc=dc.drop_duplicates()
        dc=dc.set_index(['county','date']).unstack().dropna(how='any',axis=1).stack().reset_index()
#         dc=dc.set_index(pd.to_datetime(dc['date'])).sort_index().loc['2020-12-01':].drop('date',axis=1).reset_index()
        dc=dc.set_index(pd.to_datetime(dc['date'])).sort_index().drop('date',axis=1).reset_index()
        dc['date']=dc['date'].astype(str).str[:10]
        dc=dc.dropna()
#         dc=dc.set_index(pd.to_datetime(dc['date'])).sort_index().loc['2020-12-01':].reset_index(drop=True)
        
#         dc=dc[pd.to_datetime(dc['date'])<'2020-08-10'] #DIASBLE LATER WHEN GOOGLE MOBILITY BACK TO NORMAL!
    open(htmlipath+'panels/county3ag_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))
    # open('county3_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

Mobility counties

In [206]:
dw=dn.reset_index().set_index(['date']).sort_index()['2020-02-16':].groupby('langtype')\
    .resample('1D',label='left').mean().reset_index()
dw=pd.DataFrame(np.round(dw.set_index(['date','langtype']).T.mean(),0)).reset_index()

In [207]:
dx=dw.join(dh.set_index('HU'),on='langtype').drop([1,'county'],axis=1)
dx.columns=['date','HU','value','RO','EN']
dx=pd.DataFrame(dx.set_index(['date','value']).stack().reset_index()).set_index('date')
dx.columns=['value','lang','langtype']
dx['value']=np.round(dx['value']*1.0,0)

In [87]:
drs=[]
for lang in languages:
    dr=mobility_mini.copy()#*100
    dr['lang']=lang
    dr['langtype']=szotar['ro'][lang]
    drs.append(dr)
drs=pd.concat(drs)
drs.columns=['value','lang','langtype']

In [209]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='mobility4'
push2influx(pd.concat([dx,drs],axis=0).dropna(),measurement,field_columns,tag_columns)

Slicing mobility4 from 2021-04-22 00:00:00+00:00 ...
Writing to mobility4 ...
894 data points will be written in 0.1788 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Imobiliare map

In [210]:
sheet='realEstateTransactions'
di=pd.read_csv(url+sheet, header=None)

In [211]:
di[1]=di[1].replace({'BISTRIȚA NĂSĂUD':'BISTRIȚA-NĂSĂUD','BACAU':'BACĂU',
                       'CARAȘ SEVERIN':'CARAȘ-SEVERIN','DAMBOVIȚA':'DÂMBOVIȚA','IASI':'IAȘI'})
di=di.dropna(axis=1,how='all').set_index(1)[[2,5,8,11,14,17,20,23,26,29,32,35,38,41,44]][1:-1].dropna()
di.columns=['2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01','2020-07-01',
            '2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01']
for c in di.columns:
    di[c]=di[c].str.replace(',','').astype(float)
dj=di.copy()

In [212]:
djm=(di['2020-01-01']+di['2020-02-01']+di['2020-03-01'])/3
for c in dj.columns:
    dj[c]=-100+(di[c]*100/djm)

In [213]:
djro=pd.DataFrame(dj.mean())
djro.columns=['ROMÂNIA']
dj=pd.concat([dj,djro.T])

diro=pd.DataFrame(di.sum())
diro.columns=['ROMÂNIA']
di=pd.concat([di,diro.T])

dhr=pd.concat([dh,pd.DataFrame(szotardf.loc['ro'][languages]).T])

In [214]:
di=di.join(dhr.set_index(dhr['RO'].str.upper())).set_index(languages+['county'])[di.columns].stack().reset_index()
di.columns=languages+['county','date','value']
di['date']=(pd.to_datetime(di['date'])-pd.to_timedelta('1d')).astype(str).str[:10]
for lang in languages:
    dc=di.reset_index()[['date',lang,'value','county']].dropna()
    dc.columns=['date','county','value','id']
    open(htmlipath+'panels/county4_202101'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

In [215]:
dj=dj.join(dhr.set_index(dhr['RO'].str.upper())).set_index(languages+['county'])[dj.columns].stack().reset_index()
dj.columns=languages+['county','date','value']
dj['value']=np.round(dj['value'],0)
dj['date']=(pd.to_datetime(dj['date'])-pd.to_timedelta('1d')).astype(str).str[:10]
for lang in languages:
    dc=dj.reset_index()[['date',lang,'value','county']].dropna()
    dc.columns=['date','county','value','id']
    for c in ['value']:
        dd=dc.set_index(['date',c])[['county']].sort_index(ascending=False)
        dds=[]
        for d in dd.index.get_level_values(0).unique():
            dds.append(dd.loc[[d]].iloc[:top])
            dds.append(dd.loc[[d]].iloc[-top:])
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['date','county']).join(dds.reset_index().set_index(['date','county'])[c+'_display']).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
    open(htmlipath+'panels/county4b_202102'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

In [216]:
di.join(dj['value'],rsuffix='_perc').to_csv('github/data/real_estate_transactions_ro_counties.csv')

Imobiliare counties

In [217]:
di=di.set_index(['county','date','value']).stack().reset_index().set_index('date')
di.columns=['county','value','lang','langtype']
di.index=pd.to_datetime(di.index)

In [218]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='imobiliare1'
push2influx(di,measurement,field_columns,tag_columns,fo=True)

Purging imobiliare1 ...
Writing to imobiliare1 ...
1935 data points will be written in 0.387 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [219]:
dj=dj.set_index(['county','date','value']).stack().reset_index().set_index('date')
dj.columns=['county','value','lang','langtype']
dj.index=pd.to_datetime(dj.index)

In [220]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='imobiliare2'
push2influx(dj,measurement,field_columns,tag_columns,fo=True)

Purging imobiliare2 ...
Writing to imobiliare2 ...
1935 data points will be written in 0.387 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Imobiliare cities

In [221]:
sheet='realEstatePrices'
dp0=pd.read_csv(url+sheet, header=None)

In [222]:
dp=dp0.dropna(axis=1)[1:7]
dp.columns=['county','2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01',
            '2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01']
dp2=dp0.dropna(axis=1)[8:]
dp2.columns=['county','2008-07-01','2008-08-01','2008-09-01','2008-10-01','2008-11-01',
             '2008-12-01','2009-01-01','2009-02-01','2009-03-01','2009-04-01','2009-05-01','2009-06-01','2009-07-01','2009-08-01','2009-09-01']
# dp2.columns=dp.columns

In [223]:
dp2.to_csv('github/data/real_estate_prices_ro_cities.csv')

In [224]:
dp=dp.set_index('county').T.stack().reset_index()
dp.columns=['date','county','value']
dp['date']=pd.to_datetime(dp['date'])-pd.to_timedelta('1d')
dp['date']=pd.to_datetime(dp['date'])
dp=dp.set_index('date')
dp['value']=dp['value'].astype(float)

In [225]:
dp2=dp2.set_index('county')
dp2.columns=pd.to_datetime(dp2.columns)-pd.to_timedelta('1d')
dp2=dp2.T
dp2['Year']=2008

In [226]:
dp=dp.reset_index().set_index(['date','county']).unstack()['value']
dp['Year']=2020

In [227]:
dpp=pd.concat([dp,dp2])
dpp.index=dpp.index.astype(str).str[:10]
dpp.to_excel(htmlipath+'panels/imobiliare1.xlsx')

In [228]:
dpp=dpp.set_index('Year',append=True).stack().reset_index()
dpp.columns=['date','year','city','price']
dpp['price']=dpp['price'].astype(float)
dpp['date']=pd.to_datetime(dpp['date'])
dpp=dpp.set_index('date')

In [229]:
dpp=dpp.reset_index().set_index('city').join(szotardf[languages]).set_index(['date','year','price']).stack().reset_index()
dpp.columns=['date','year','price','lang','city']
dpp['cityyear']=dpp['city']+' '+dpp['year'].astype(str)

In [230]:
dpp=dpp.set_index('date')

In [231]:
field_columns=['price']
tag_columns=['city','year','lang','cityyear']
measurement='imobiliare3'
push2influx(dpp,measurement,field_columns,tag_columns,dbclient=client_long,fo=True)

Purging imobiliare3 ...
Writing to imobiliare3 ...
540 data points will be written in 0.108 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Imobiliare cities map

In [232]:
coords={'Brasov':'45.64861, 25.60613', 
        'Bucharest':'44.43225, 26.10626', 
        'Cluj':'46.76667, 23.6', 
        'Constanta':'44.18073, 28.63432', 
        'Timisoara':'45.75372, 21.22571',
        'Max':'0,0'}
popo={'Brasov':276088, 
        'Bucharest':1877155, 
        'Cluj':316748, 
        'Constanta':303399, 
        'Timisoara':315053,
        'Max':2000000}
pop={'Brasov':276088, 
        'Bucharest':500000, 
        'Cluj':316748, 
        'Constanta':303399, 
        'Timisoara':315053,
        'Max':500000}

In [233]:
dp2=dp2[:1].T
dp2.columns=['2019-11-30']
dp=dp.T.join(dp2).T
dp['Max']=2200

In [234]:
do=dp.T.join(pd.DataFrame(coords,index=['coords']).T).dropna().join(pd.DataFrame(pop,index=['pop']).T).\
    set_index(['coords','pop'],append=True).stack().reset_index()
do['lat']=do['coords'].str.split(',').str[0].str.strip().astype(float)
do['lon']=do['coords'].str.split(',').str[1].str.strip().astype(float)
do.columns=['county','coords','pop','date','price','lat','lon']
do['date']=do['date'].astype('str').str[:10]

In [235]:
for lang in languages:
    dc=do.copy()
    dc['county']=dc['county'].replace(szotardf[lang].to_dict())
    open(htmlipath+'panels/county4c_202102'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

Matrix

In [236]:
sheet='matrix'
df=pd.read_csv(url+sheet)[:-1]
df=df[['IND HU short','IND RO short','IND EN short','% of GDP (2018)',
       'GDP gain/loss in 2020 Q2 (%)','GDP gain/loss in 2020 Q3 (%)']]
df.columns=['HU','RO','EN','share','q2','q3']
df['share']=df['share'].str.replace('%','').astype(float)
for q in ['q2','q3']:
    df[q]=df[q].str.replace('%','').astype(float)
# df=pd.DataFrame(df.set_index(languages+['share']).stack()).reset_index()
# df.columns=languages+['share','quarter','value']
# df['group']=1

In [237]:
df.to_csv('github/data/sectoral_effects_industry.csv')

In [238]:
for lang in languages:
    dc=df[[lang,'share','q2','q3']]
    dc.columns=['id','x','q2','q3']
    dc['c2']=(dc['x']/dc['x'].median())*dc['q2']
    dc['c3']=(dc['x']/dc['x'].median())*dc['q3']
    open(htmlipath+'panels/matrix_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))
    # open('matrix_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

<ipython-input-238-86558fe078b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc['c2']=(dc['x']/dc['x'].median())*dc['q2']
<ipython-input-238-86558fe078b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc['c2']=(dc['x']/dc['x'].median())*dc['q2']
<ipython-input-238-86558fe078b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [239]:
ds=szotardf[languages].loc[['% of GDP (2018)','GDP gain/loss in 2020 Q2 (%)']].T
ds.columns=['x','y']
open(htmlipath+'panels/matrix_label.json','w').write(json.dumps(ds.to_dict()))
#open('matrix_label.json','w').write(json.dumps(ds.to_dict()))

305

Severity map

In [59]:
# By Jake VanderPlas
# License: BSD-style

import matplotlib.pyplot as plt
import numpy as np


def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [60]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import to_hex
cm = LinearSegmentedColormap.from_list(
        'my_cmap', ['gold','orange','crimson','darkRed'] , N=8)

In [61]:
severity['month']=severity.index.month

In [62]:
sv_label=szotardf.loc[['severity_plot2x','severity_plot2y','severity_plot2s',
                      'severity_q1','severity_q2','severity_q3','severity_q4'
                      ]][languages]
sv_label.index=['x','y','s','q1','q2','q3','q4']
open(htmlipath+'panels/severity_label2.json','w').write(json.dumps(sv_label.T.to_dict()))

1444

In [89]:
for lang in languages:
    smo=['2020-02-28','2020-03-31','2020-04-30','2020-05-31','2020-06-30','2020-07-31','2020-08-31',
         '2020-09-30','2020-10-31','2020-11-30','2020-12-31','2021-01-31','2021-02-28','2021-03-31','2021-04-30',str(severity.index.max())[:10]]
    sv=df0.loc[smo]['cases'].astype(int)
    svd=sv.diff()
    svd.loc['2020-02-28']=sv.loc['2020-02-28']
    svd.name=sv_label.loc['s'][lang]

    severity['month']=severity.index.strftime('%Y-%m')
    sy=severity[((severity['type']=='stringency')&(severity['lang']==lang)&(severity['lang2']==lang))]\
        .drop(['country','langcountry','lang','lang2','type','langtype'],axis=1).groupby('month').mean()
    sy.index=smo
    sy=sy['value']
    sy.name=sv_label.loc['y'][lang]
    sy.index=pd.to_datetime(sy.index)

    svy=pd.DataFrame(svd).join(sy)
    svy.index=pd.to_datetime(svy.index)
    svy['month']=svy.index.strftime('%Y-%m')

    sm=pd.concat([dx,drs],axis=0)
    sm=sm[sm['langtype']==szotar['ro'][lang]]
    sm['month']=sm.index.strftime('%Y-%m')
    sm=-sm.groupby('month').mean()['value']
    sm.name=sv_label.loc['x'][lang]
    svy=svy.set_index('month').join(sm).reset_index()
    svy['color']=[to_hex(cm(i/max(svy[sv_label.loc['s'][lang]]))) for i in svy[sv_label.loc['s'][lang]]]
    #     svy=pd.concat([svy,pd.DataFrame([{'month':0,sv_label.loc['x'][lang]:20,sv_label.loc['y'][lang]:20,sv_label.loc['s'][lang]:1,'color':'#000000'}])])
    #     svy.loc[0,sv_label.loc['s'][lang]]=1000
    #     svy=svy.dropna() #also implies deleting A9 on the list below
    svy=svy.ffill()
    svy1=svy[((pd.to_datetime(svy['month'])>=pd.to_datetime('2020-04'))&((pd.to_datetime(svy['month'])<=pd.to_datetime('2020-08'))))]\
        .sort_values(by='month',ascending=False)[[sv_label.loc['x'][lang],sv_label.loc['y'][lang]]]
    svy1=pd.concat([pd.DataFrame({sv_label.loc['x'][lang]: 42, sv_label.loc['y'][lang]: 89},index=[98]),
                    pd.DataFrame({sv_label.loc['x'][lang]: 25, sv_label.loc['y'][lang]: 64},index=[96]),
                    pd.DataFrame({sv_label.loc['x'][lang]: 6, sv_label.loc['y'][lang]: 45},index=[94]),
                    pd.DataFrame({sv_label.loc['x'][lang]: 0, sv_label.loc['y'][lang]: 39},index=[93]),
                    pd.DataFrame({sv_label.loc['x'][lang]: 16, sv_label.loc['y'][lang]: 51},index=[95]),svy1])
    svy1['line']='A'
    svy2=svy[((pd.to_datetime(svy['month'])>=pd.to_datetime('2020-02'))&((pd.to_datetime(svy['month'])<=pd.to_datetime('2020-04'))))]\
        .sort_values(by='month',ascending=True)[[sv_label.loc['x'][lang],sv_label.loc['y'][lang]]]
    svy2=pd.concat([pd.DataFrame({sv_label.loc['x'][lang]: 41, sv_label.loc['y'][lang]: 75},index=[99]),svy2])
    svy2['line']='B'
    svy3=svy[((pd.to_datetime(svy['month'])>=pd.to_datetime('2020-08'))&((pd.to_datetime(svy['month'])<pd.to_datetime('2021-02'))))]\
        .sort_values(by='month',ascending=True)[[sv_label.loc['x'][lang],sv_label.loc['y'][lang]]]
    svy3=pd.concat([pd.DataFrame({sv_label.loc['x'][lang]: 19, sv_label.loc['y'][lang]: 61},index=[97]),svy3])
    svy3['line']='C'
    svy4=svy[((pd.to_datetime(svy['month'])>=pd.to_datetime('2021-01'))&((pd.to_datetime(svy['month'])<pd.to_datetime('2021-06'))))]\
        .sort_values(by='month',ascending=True)[[sv_label.loc['x'][lang],sv_label.loc['y'][lang]]]
    svy4=pd.concat([pd.DataFrame({sv_label.loc['x'][lang]: 19, sv_label.loc['y'][lang]: 81},index=[92]),svy4])
    svy4['line']='D'
    open(htmlipath+'panels/severity1_5_'+lang+'.json','w').write(json.dumps(
        {'circles':list(np.round(svy,1).reset_index().T.to_dict().values()),
        'lines1b':list(svy1.T.to_dict().values()),
        'lines2b':list(svy2.T.to_dict().values()),
        'lines3b':list(svy3.T.to_dict().values()),
         'lines4b':list(svy4.T.to_dict().values())
        }))
    print(lang)

HU
RO
EN


Macro

Restart `InfluxDB` and `Grafana` to free up memory

In [273]:
icons={'YoY%':'📈', '%':'%', 'proxy index':'➰', 'value index':'💲',
       'volume index':'🛒', 'index':'〽'}

In [274]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-35-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-35-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-35-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [275]:
sheet='EcMonitor'
df=pd.read_csv(url+sheet,header=None).dropna(how='all',axis=1).set_index(0)
df.index.name='indicator'
df.loc['GDP']=df.loc['GDP'].bfill()
df.loc['Market consensus']=df.loc['Market consensus'].bfill()
df['order']=[100*i for i in range(len(df.index))]
df=df.set_index('order',append=True)
df=df.drop(['Indicator'],axis=0,level=0)
df2=df[1:].drop([1],axis=1)
# df2=df2.dropna(how='all')
sections=[i[0] for i in df[df.isnull().all(axis=1)].index]
df[1]=df[1].fillna('row')
# df=df[df[1:].dropna(how='all',axis=1).columns]
df[1]=df[1].replace(icons)
df1=df.stack().dropna().reset_index()
df1.columns=['indicator','order','index','value']
df2=df2.astype(float)

In [276]:
cscaler=1
colors=df2.reset_index().set_index('indicator').T.count().to_dict()
colors={i:{'size':colors[i]} for i in colors}
for i in df2.index:
    colors[i[0]]['min']=float(df2.loc[i[0]].min().min()*cscaler)
    colors[i[0]]['max']=float(df2.loc[i[0]].max().max()*cscaler)

In [277]:
colorlist=[]
for i in df1[['indicator','value','index']].T.iteritems():
    if i[1][2]==2:
        if i[1][0] not in ['date']:
            ref_color=float(i[1][1])
            if i[1][0] in ['GDP','Market consensus','DFM estimate','No. of employees']:
                ref_color=0
            if i[1][0] in ['Construction works']:
                ref_color=20
            if i[1][0] in ['Consumer price index']:
                ref_color=3
            if i[1][0] in ['Non-government loans']:
                ref_color=6
            if i[1][0] in ['Non-government deposits']:
                ref_color=10.7
            color='transparent'
    elif i[1][2]==1:
        color='transparent'
    elif i[1][0] in ['date']:
        color='transparent'
    elif i[1][0] not in colors:
        color='transparent'
    else:
        #if i[1][0] in ['GDP','Market consensus','% of information included in the model']:
        if i[1][0] in ['% of information included in the model']:
            cmap='Blues'
        else:
            cmap='RdBu'
        rc=(ref_color*1.0-colors[i[1][0]]['min'])*1.0/(colors[i[1][0]]['max']*1.0-colors[i[1][0]]['min']*1.0)
        c=(float(i[1][1])*1.0-colors[i[1][0]]['min'])*1.0/(colors[i[1][0]]['max']*1.0-colors[i[1][0]]['min']*1.0)
        if rc==0: rc=0.1
        c=c*0.5/rc
        if (i[1][0] in ['Unemployment ratio']):
            c=1-c
        # color=adjust_lightness(discrete_cmap(colors[i[1][0]]['size'],cmap)(c),1)
        color=adjust_lightness(discrete_cmap(20,cmap)(c),1)
    colorlist.append(color)

In [278]:
valuelist=[]
for i in df1[['indicator','value','index']].T.iteritems():
    if i[1][2]==1:
        valuelist.append(i[1][1])
    elif i[1][0] in colors:
        if float(i[1][1])<100:
            valuelist.append(str(np.round(float(i[1][1]),1)).replace('.0',''))
        else:
            valuelist.append(str(int(np.round(float(i[1][1]),0))))
    else:
        valuelist.append(i[1][1])

In [279]:
df1['color']=colorlist
df1['value']=valuelist

In [280]:
sets={'groups':[],'lang':szotardf.loc[list(df.droplevel('order')[2].index)+['data','desc','type','month']][:].T.to_dict()}
sets['lang']={i:{k:sets['lang'][i][k] for k in sets['lang'][i] if str(sets['lang'][i][k]) not in ['nan','XXX']} for i in sets['lang']}
for i in df.droplevel('order')[1:][2].iteritems():
    if str(i[1])=='nan':
        #sets['groups'].append([])
        sets['groups'].append([i[0]])
    else:
        sets['groups'][-1].append(i[0])
# sets[0]=[item for sublist in sets['groups'] for item in sublist]

In [281]:
sets_months={'2015-2019':{'HU':'2015-2019','RO':'2015-2019','EN':'2015-2019'},
            '2019-06':{'HU':'2019 Jún','RO':'2019 Iun','EN':'2019 Jun'},
             '2019-07':{'HU':'Júl','RO':'Iul','EN':'Jul'},
            '2019-08':{'HU':'Aug','RO':'Aug','EN':'Aug'},
             '2019-09':{'HU':'2019 Sze','RO':'2019 Sep','EN':'2019 Sep'},
             '2019-10':{'HU':'Okt','RO':'Oct','EN':'Oct'},
             '2019-11':{'HU':'Nov','RO':'Noi','EN':'Nov'},
             '2019-12':{'HU':'Dec','RO':'Dec','EN':'Dec'},
             '2020-01':{'HU':'2020 Jan','RO':'2020 Ian','EN':'2020 Jan'},
             '2020-02':{'HU':'Feb','RO':'Feb','EN':'Feb'},
             '2020-03':{'HU':'Már','RO':'Mar','EN':'Mar'},
             '2020-04':{'HU':'Ápr','RO':'Apr','EN':'Apr'},
             '2020-05':{'HU':'Máj','RO':'Mai','EN':'May'},
             '2020-06':{'HU':'Jún','RO':'Iun','EN':'Jun'},
             '2020-07':{'HU':'Júl','RO':'Iul','EN':'Jul'},
             '2020-08':{'HU':'Aug','RO':'Aug','EN':'Aug'},
             '2020-09':{'HU':'Sze','RO':'Sep','EN':'Sep'},
            '2020-10':{'HU':'Okt','RO':'Oct','EN':'Oct'},
            '2020-11':{'HU':'Nov','RO':'Noi','EN':'Nov'},
            '2020-12':{'HU':'Dec','RO':'Dec','EN':'Dec'},
            '2021-01':{'HU':'2021 Jan','RO':'2021 Ian','EN':'2021 Jan'},
             '2021-02':{'HU':'Feb','RO':'Feb','EN':'Feb'},
             '2021-03':{'HU':'Már','RO':'Mar','EN':'Mar'},
             }
sets_months2={
            '2019-06':{'HU':'Június','RO':'Iunie','EN':'June'},
             '2019-07':{'HU':'Július','RO':'Iulie','EN':'July'},
            '2019-08':{'HU':'Augusztus','RO':'August','EN':'August'},
             '2019-09':{'HU':'Szeptember','RO':'Septembrie','EN':'September'},
             '2019-10':{'HU':'Október','RO':'Octombrie','EN':'October'},
             '2019-11':{'HU':'November','RO':'Noiembrie','EN':'November'},
             '2019-12':{'HU':'December','RO':'Decembrie','EN':'December'},
             '2020-01':{'HU':'Január','RO':'Ianuarie','EN':'January'},
             '2020-02':{'HU':'Február','RO':'Februarie','EN':'February'},
             '2020-03':{'HU':'Március','RO':'Martie','EN':'March'},
             '2021-01':{'HU':'Január','RO':'Ianuarie','EN':'January'},
             '2021-02':{'HU':'Február','RO':'Februarie','EN':'February'},
             '2021-03':{'HU':'Március','RO':'Martie','EN':'March'},
             '2020-04':{'HU':'Április','RO':'Aprilie','EN':'April'},
             '2020-05':{'HU':'Május','RO':'Mai','EN':'May'},
             '2020-06':{'HU':'Június','RO':'Iunie','EN':'June'},
             '2020-07':{'HU':'Július','RO':'Iulie','EN':'July'},
             '2020-08':{'HU':'Augusztus','RO':'August','EN':'August'},
             '2020-09':{'HU':'Szeptember','RO':'Septembrie','EN':'September'},
            '2020-10':{'HU':'Október','RO':'Octombrie','EN':'October'},
            '2020-11':{'HU':'November','RO':'Noiembrie','EN':'November'},
            '2020-12':{'HU':'December','RO':'Decembrie','EN':'December'}}
sets_months['type']=szotardf.loc['type'][languages].to_dict()
sets['months']={i:sets_months[i] for i in df.loc['date'].values[0]}
sets_labels=df[:1].T['date'].to_dict()[0]
sets_labels={i:{j:sets_labels[i].split('-')[0]+' '+sets_months2[sets_labels[i]][j] \
    for j in sets_months2[sets_labels[i]]} for i in sets_labels if sets_labels[i] in sets_months2}
sets_labels['indicator']=szotardf.loc['indicator'][languages].to_dict()
sets_labels[2]={'HU':'2015-2019 átlag','RO':'medie 2015-2019','EN':'2015-2019 average'}
sets['label']=sets_labels

In [282]:
open(htmlipath+'panels/macro_sets79.json','w').write(json.dumps(sets))

20825

In [283]:
df3=df1.join(pd.DataFrame(sets['label']).T,on='index')
df4=df3[['value']+languages].T.ffill()
for lang in languages:
    df3[lang]=df4.T[lang].replace({icons[i]:szotar[i][lang] for i in icons})
open(htmlipath+'panels/macro_table80.json','w').write(json.dumps(list(df3.T.to_dict().values())))

110676

Macro scatter

In [254]:
# process_macro=True

In [255]:
# client_long.drop_database(dbname_long)
# client_long.drop_retention_policy(dbname_long)
# client_long.create_database(dbname_long)
# client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [256]:
if process_macro:
    sheet='DFMChartYoY'
    df=pd.read_csv(url+sheet)
    df=df.set_index('Date').loc[['DFM estimate','GDP','GDP.1','Market consensus','Range68']].T.\
        dropna(how='all',axis=0).astype(float).reset_index()
    df.index=[(pd.to_datetime('now')-pd.to_timedelta('10H')*(1+i)).strftime('%Y-%m-%d-%H') for i in range(len(df))][::-1]
    df.index=pd.to_datetime(df.index)
    df['index']=pd.to_datetime(df['index'].str.replace('-','-20'))
    df['CI+']=df['DFM estimate']+df['Range68']/2
    df['CI-']=df['DFM estimate']-df['Range68']/2
    df['CI++']=df['DFM estimate']+df['Range68']
    df['CI--']=df['DFM estimate']-df['Range68']
    df=pd.DataFrame(df.set_index('index').stack()).reset_index()
    df.columns=['date','type','value']
    df=df.set_index('date')
    df['value']=df['value'].astype(float)
    df=df.join(szotardf,on='type')
    df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
    df.columns=['type','value','lang','langtype']
    df.index=pd.to_datetime(df.index)+pd.DateOffset(months=1)-pd.DateOffset(days=1)

In [257]:
if process_macro:
    field_columns=['value']
    tag_columns=['lang','type','langtype']
    measurement='macro'
    push2influx(df,measurement,field_columns,tag_columns,dbclient=client_long,fo=True)

Purging macro ...
Writing to macro ...
5652 data points will be written in 1.1304 batches.
Expected query running time is: 5 seconds.
Writing batch 1 ...
Writing batch 2 ...
Done!


Calculate data complexity measures

In [258]:
measurements=client.query('show measurements')
measurements_long=client_long.query('show measurements')

In [259]:
ms={}
for m in [j['name'] for j in [i for i in measurements][0]]:
    print(m)
    time.sleep(1)
    ms[m]=client.query('SELECT COUNT(*) FROM '+m)
for m in [j['name'] for j in [i for i in measurements_long][0]]:
    print(m)
    time.sleep(1)
    ms[m]=client_long.query('SELECT COUNT(*) FROM '+m)

cats
counties
counties2
counties3
firms1
firms2
firms3
firms4
forecast
global2
global3
global4
governance1
governance2
governance3
imobiliare1
imobiliare2
mobility
mobility2
mobility3
mobility4
news1
registry1
severity
social1
social2
social3
stocks1
stocks2
vaccine1
imobiliare3
macro


In [260]:
l=len(languages)
cat={'status':['governance1','governance2','counties','counties2','counties3']+['governance3']*3,
    'global':['global3','global4']+['global2']*l,
    'governance':['social1','social2','social3']*l+['severity'],
    'news':['news1']*l*2,
    'macro':['macro'],
    'stocks':['stocks1','stocks2'],
    'firms':['firms1','firms4']+['firms2','firms3']*l,
    'industry':[],
    'mobility':['mobility','mobility2','mobility3','mobility4'],
    'imobiliare':['imobiliare1','imobiliare2','imobiliare3']}
mg={i:0 for i in cat}
ms2={}
for m in ms:
    for d in ms[m]:
        ms2[d]=ms[m][d].sum().sum()
for c in cat:
    for m in cat[c]:
        mg[c]+=ms2[m]
mg['mobility']/=l

In [261]:
cat2={'macro':['macro_table','macro_sets'],
    'industry':['matrix_label']+['matrix_HU']*3+['county_HU']*3,
    'mobility':['county3ad_HU']*3*2,
    'imobiliare':['county4c_202012HU']*3*4+['county4b_202012HU']*3*2+['county4_202012HU']*3*2}

In [262]:
for c in cat2:
    for m in cat2[c]:
        mg[c]+=len(json.loads(open(htmlipath+'panels/'+m+'.json','r').read()))

In [263]:
import os, os.path
cat2['status']=len([name for name in os.listdir(htmlipath+'panels/daily/')])*len(json.loads(open(htmlipath+'panels/daily/county2b_2021-01-01_HU.json','r').read()))*5

In [264]:
open(htmlipath+'panels/cats.json','w').write(json.dumps(list(\
     pd.DataFrame(mg,index=['count']).T.join(szotardf[languages]).T.to_dict().values())))

1176

In [265]:
df=pd.DataFrame(mg,index=['count']).T.join(szotardf[languages]).set_index('count').stack()\
                                                              .reset_index()

In [266]:
open(htmlipath+'panels/cats2.json','w').write(json.dumps(list(\
     df.T.to_dict().values())))

2052

In [267]:
df['date']=pd.to_datetime('2020-04-05')
df=df.set_index('date')
df.columns=['count','lang','langtype']

In [268]:
field_columns=['count']
tag_columns=['lang','type','langtype']
measurement='cats'
push2influx(df,measurement,field_columns,tag_columns,fo=True)

Purging cats ...
Writing to cats ...
30 data points will be written in 0.006 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [269]:
sum(mg.values())

537846.0

In [270]:
print(pd.to_datetime('now'))

2021-05-02 16:05:30.572931
